# HF Transformers 核心模块学习：Pipelines 进阶

我们已经完成了 Pipeline API 针对各类任务的基本使用。

实际上，在 Transformers 库内部实现中，Pipeline 作为管理：`原始文本-输入Token IDs-模型推理-输出概率-生成结果` 的流水线抽象，背后串联了 Transformers 库的核心模块 `Tokenizer`和 `Models`。

![](docs/images/pipeline_advanced.png)

下面我们开始结合大语言模型（在 Transformers 中也是一种特定任务）学习：

- 使用 Pipeline 如何与现代的大语言模型结合，以完成各类下游任务
- 使用 Tokenizer 编解码文本
- 使用 Models 加载和保存模型

Hugging Face Transformers 库中的 **Pipeline（管道）** 是一个高度封装的工具，可以理解为一条“自动化流水线”。它把自然语言处理任务的复杂流程（如文本分词、模型推理、结果解码等）打包成简单的接口，让用户无需深入底层代码即可完成各种任务。以下用通俗语言分模块解释其进阶原理和应用：

---

### 一、Pipeline 的“流水线”设计原理
想象你要用大语言模型（如 GPT、LLaMA）完成一个任务，流程通常是：  
1. **文本输入** → 2. **分词处理** → 3. **模型计算** → 4. **结果解码**  
Pipeline 将这些步骤串联起来，形成一条“流水线”，具体分工如下：
- **Tokenizer（分词器）**：将原始文本拆分为模型能理解的 Token（如单词或子词），并转换为数字 ID。例如，句子 `"Hello world!"` 会被拆解为 `[101, 7592, 2088, 102]`（BERT 的 Token ID）。
- **Model（模型）**：接收 Token ID，通过神经网络计算生成原始输出（如概率分布或生成文本）。
- **Post-Processing（后处理）**：将模型的原始输出转化为人类可读的结果。例如，情感分析任务会将概率最高的标签（如“积极/消极”）返回。

> **示例**：用 Pipeline 生成文本时，只需一行代码：  
> ```python
> generator = pipeline("text-generation", model="meta-llama/Llama-2-7b")
> result = generator("今天的天气真不错，我打算")
> ```
> 背后自动完成：分词 → 模型生成 → 解码为中文文本。

---

### 二、与大语言模型结合的核心技巧
现代大语言模型（如 LLaMA、Qwen）通常以生成式任务为主，Pipeline 提供了灵活的参数控制：
1. **生成参数控制**：  
   通过 `max_length`（生成最大长度）、`temperature`（随机性）、`top_k`（候选词数量）等参数调整生成结果。例如：
   ```python
   generator("写一首关于春天的诗", max_length=100, temperature=0.7)
   ```
2. **本地模型加载**：  
   支持直接加载本地模型文件（如从 `D:/ModelSpace` 目录），无需联网下载：
   ```python
   model = AutoModelForCausalLM.from_pretrained("本地模型路径", device_map="auto")
   pipeline("text-generation", model=model)
   ```
3. **多模态扩展**：  
   Pipeline 不仅支持文本任务，还能处理音频分类、语音识别（如加载 `facebook/wav2vec2` 模型）、图像分类等。

---

### 三、Tokenizer 的编解码奥秘
Tokenization 是 Pipeline 的关键步骤，直接影响模型表现：
- **编码（Encode）**：将文本 → Token ID。例如，BERT 分词器会添加特殊符号（如 `[CLS]`、`[SEP]`）表示句子边界。
- **解码（Decode）**：将 Token ID → 文本。例如，生成任务中模型输出的 ID 序列会被还原为连贯的句子。

> **注意**：不同模型的分词方式不同（如 BPE 算法、子词切分），需使用对应的 Tokenizer。

---

### 四、Models 的加载与优化
1. **自动选择框架**：  
   Transformers 支持 PyTorch（`pt`）和 TensorFlow（`tf`），Pipeline 会根据环境自动选择模型类型。
2. **设备映射（Device Mapping）**：  
   通过 `device_map="auto"`，模型可自动分配到 GPU 或 CPU，甚至支持多卡并行。
3. **性能优化**：  
   - **批处理（Batch）**：一次性处理多个输入提升效率（如 `batch_size=8`）。
   - **量化（Quantization）**：将模型权重压缩为低精度格式（如 8 位），减少内存占用。

---

### 五、总结：Pipeline 的适用场景
- **快速原型开发**：一行代码调用预训练模型。
- **教育/演示**：无需关注底层细节，直观展示模型能力。
- **生产部署**：通过性能优化（批处理、量化）支持高并发。

> **局限性**：若需要自定义模型结构或复杂后处理逻辑，仍需直接调用 `Tokenizer` 和 `Model` 类。

通过 Pipeline，开发者可以像搭积木一样组合不同任务（如“先做情感分析，再生成回复”），极大降低了 NLP 和大模型的应用门槛。

## 使用 Pipeline 调用大语言模型

### Language Modeling

语言建模是一项预测文本序列中的单词的任务。它已经成为非常流行的自然语言处理任务，因为预训练的语言模型可以用于许多其他下游任务的微调。最近，对大型语言模型（LLMs）产生了很大兴趣，这些模型展示了零或少量样本学习能力。这意味着该模型可以解决其未经明确训练过的任务！虽然语言模型可用于生成流畅且令人信服的文本，但需要小心使用，因为文本可能并不总是准确无误。

通过理论篇学习，我们了解到有两种典型的语言模型：

- 自回归：模型目标是预测序列中的下一个 Token（文本），训练时对下文进行了掩码。如：GPT-3。
- 自编码：模型目标是理解上下文后，补全句子中丢失/掩码的 Token（文本）。如：BERT。


以下是使用Hugging Face Transformers库中的`pipeline`调用大语言模型进行语言建模的实践指南，结合自回归（如GPT-3）和自编码（如BERT）两种典型模型的特性：

---

### 一、Pipeline核心原理与配置
Hugging Face的`pipeline`通过三步完成推理：文本预处理→模型推理→后处理输出。调用大语言模型需明确任务类型：
```python
from transformers import pipeline

# 自回归模型（文本生成）
generator = pipeline("text-generation", model="gpt2")  # 支持GPT-3需API密钥

# 自编码模型（语义理解）
mask_filler = pipeline("fill-mask", model="bert-base-uncased")
```

---

### 二、自回归模型应用（生成式任务）
**适用场景**：文本续写、对话生成、代码补全  
**代码示例**：
```python
# 生成多样化的文本
outputs = generator(
    "Artificial intelligence will", 
    max_length=100,
    num_return_sequences=3,
    temperature=0.9,  # 控制随机性（0~1）
    top_k=50          # 限制候选词范围
)
for result in outputs:
    print(result["generated_text"])
```
**关键参数说明**：
- `max_length`：生成文本的最大长度
- `temperature`：值越高生成越随机（>1可能产生乱码）
- `repetition_penalty`：抑制重复词生成（建议1.2-2.0）

---

### 三、自编码模型应用（理解式任务）
**适用场景**：文本修复、语义分析、实体识别  
**代码示例**：
```python
# 补全掩码文本
masked_text = "Hugging Face is a [MASK] company based in New York."
results = mask_filler(masked_text, top_k=5)

for res in results:
    print(f"候选词：{res['token_str']}，置信度：{res['score']:.4f}")
```
**输出示例**：
```
候选词：technology，置信度：0.8721  
候选词：software，置信度：0.1234  
...（显示前5个预测结果）
```

---

### 四、模型选择与进阶技巧
| 模型类型       | 代表模型       | 典型任务               | Pipeline参数         |
|----------------|----------------|------------------------|----------------------|
| 自回归         | GPT-3、GPT-2   | 文本生成、问答         | `text-generation`    |
| 自编码         | BERT、RoBERTa  | 掩码填充、文本分类     | `fill-mask`、`ner`   |
| 序列到序列     | BART、T5       | 摘要、翻译             | `summarization`      |

**优化建议**：
1. **硬件加速**：添加`device=0`参数启用GPU加速（需安装对应CUDA驱动）
2. **量化压缩**：使用`model="facebook/opt-125m"`等量化模型减少显存占用
3. **混合任务**：结合`pipeline(task="text2text-generation", model="t5-base")`实现跨任务处理

---

### 五、注意事项与最佳实践
1. **伦理风险**：生成的文本可能包含偏见或错误信息，建议添加内容过滤层
2. **显存管理**：大型模型（如GPT-3）需使用`device_map="auto"`自动分配多GPU资源
3. **零样本学习**：通过prompt工程实现无监督任务适配（示例见Hugging Face官方文档）

通过合理选择模型架构和调整参数，开发者可快速实现从简单文本生成到复杂语义理解的全场景覆盖。建议结合Hugging Face Hub探索超过20,000个预训练模型。

### 使用 GPT-2 实现文本生成

![](docs/images/gpt2.png)

模型主页：https://huggingface.co/gpt2

这段代码使用Hugging Face Transformers库调用GPT-2模型，根据给定的提示文本自动生成后续内容。具体解释如下：

1. **导入pipeline模块**  
   `from transformers import pipeline`  
   Hugging Face的`pipeline`是一个高级API，封装了数据预处理、模型调用和结果后处理的完整流程。通过它可以用极简代码实现复杂的NLP任务。

2. **定义输入提示**  
   `prompt = "Hugging Face is a community-based open-source platform for machine learning."`  
   这是模型生成文本的起始语句，GPT-2会根据这个上下文预测后续内容。

3. **创建文本生成器**  
   `generator = pipeline(task="text-generation", model="gpt2")`  
   这里指定了任务类型为`text-generation`，并加载了GPT-2预训练模型。该模型基于Transformer架构，擅长生成连贯的文本。若未指定模型，pipeline会默认选择适合该任务的最新模型。

4. **执行文本生成**  
   `generator(prompt)`  
   该调用会完成以下步骤：
   - **分词**：将输入文本转换为GPT-2能理解的token序列
   - **模型推理**：GPT-2通过自注意力机制预测下一个token的概率分布
   - **解码**：采用贪婪搜索或束搜索（beam search）策略生成文本
   - **输出**：返回包含生成文本的列表，默认生成20个token（可通过`max_length`参数调整）

**示例输出可能结果**：  
`[{'generated_text': 'Hugging Face is a community-based open-source platform for machine learning. It provides access to thousands of pre-trained models and datasets, allowing developers to easily implement state-of-the-art AI applications...'}]`

**扩展说明**：
- 可通过添加参数控制生成效果，例如`max_length=50`限制生成长度，`num_return_sequences=3`获取多个候选结果
- GPT-2是自回归模型，每次生成一个token并循环迭代，直到达到指定长度或遇到终止符
- 首次运行时会自动下载约500MB的模型文件到缓存目录（可通过`TRANSFORMERS_CACHE`环境变量修改路径）

In [2]:
from transformers import pipeline

prompt = "Hugging Face is a community-based open-source platform for machine learning."
generator = pipeline(task="text-generation", model="gpt2")
generator(prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hugging Face is a community-based open-source platform for machine learning. Our goal is to bring high-level user experiences, including performance, to your own and other team's needs. As people from some of the nicest companies on the"}]

In [3]:
prompt = "Introduce GitHub."
generator = pipeline(task="text-generation", model="gpt2")
generator(prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Introduce GitHub.com to use code from this repository. I'd welcome them, you'll get nice access to code samples (as far I can see) and help with my work. :-) We want to thank welder@tumontower"}]

#### 设置文本生成返回条数

In [7]:
prompt = "You are very smart"
generator = pipeline(task="text-generation", model="gpt2", num_return_sequences=3)
generator(prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'You are very smart, you are very good with music, and I don\'t give a shit about your career. I got everything."\n\nNow, the two stars are gearing up to play the same role together.\n\n"This is my'},
 {'generated_text': 'You are very smart."'},
 {'generated_text': 'You are very smart."\n\nI\'m sure everyone in here knew this was how he reacted. He is a nice guy with a good heart -- but I\'ve talked before about how he should be seen as a smart man, but he has been'}]

In [8]:
prompt = "You are very stupid"
generator = pipeline(task="text-generation", model="gpt2", num_return_sequences=5)

In [9]:
generator(prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'You are very stupid, you think we should go ahead and bring you a meal?"\n\nHe shook his head. "It\'s nothing. It\'s just a joke."\n\n"What kind of jokes?"\n\n"Oh don\'t laugh'},
 {'generated_text': 'You are very stupid, because you don\'t know what she does or if she cares to do it. You didn\'t understand."\n\n"Are you serious?"\n\nShake your head in disbelief. For a moment he wanted to know more'},
 {'generated_text': 'You are very stupid. You are very smart. You are very smart. So, I have a lot of money. And of course, you have very good intentions. I do not wish that you were living. I do not wish that I had'},
 {'generated_text': "You are very stupid, and would be a nice boy to hold in your house. What is your plan for my parents, what you need for your whole life?\n\nWhat will you do? Your plan isn't simple, it is highly destructive"},
 {'generated_text': 'You are very stupid! You are stupid!" says someone in a whisper. He then walks toward the door, and the 

In [10]:
generator(prompt, num_return_sequences=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'You are very stupid!"\n\nA number of people expressed outrage at the comments, including Dr Peter Danczuk, chair of the UK Human Rights Commission, who described the comments as "disgusting".\n\nThe comments, which were'},
 {'generated_text': 'You are very stupid and I will use all of my power to prevent you!" ―Femme Fatale\'s self-deception as Balthazar, "The Dark Side"\n\nIf Balthazar had had an idea of the Dark'}]

#### 设置文本生成最大长度

In [5]:
generator(prompt, num_return_sequences=2, max_length=16)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'You are very smart, are your hands so good, why not make a movie'},
 {'generated_text': 'You are very smart and incredibly well supported in the community and I was very nervous'}]

### 使用 BERT-Base-Chinese 实现中文补全

![](docs/images/bert-base-chinese.png)

模型主页：https://huggingface.co/bert-base-chinese

这段代码使用 **BERT-Base-Chinese** 模型实现中文文本的掩码（MASK）补全任务。以下是逐行解析和原理说明：

---

### 代码解释
1. **导入 Pipeline 模块**  
   ```python
   from transformers import pipeline
   ```
   - 从 Hugging Face Transformers 库中导入 `pipeline`，这是调用预训练模型的核心接口。

2. **创建掩码填充任务实例**  
   ```python
   fill_mask = pipeline(task="fill-mask", model="bert-base-chinese")
   ```
   - `task="fill-mask"`：指定任务类型为**掩码填充**，这是 BERT 模型的典型任务（自编码模型）。
   - `model="bert-base-chinese"`：加载中文版 BERT 预训练模型（12层Transformer，110M参数）。

---

### 核心原理
BERT 的 `fill-mask` 任务旨在预测文本中被遮盖部分（`[MASK]`）最可能出现的词语。流程如下：
1. **输入格式**：文本中必须包含 `[MASK]` 符号，表示需要预测的位置。  
2. **分词与编码**：BERT 分词器将中文文本拆分为子词（subword），并转换为 Token ID。  
3. **模型预测**：BERT 通过双向注意力机制，结合上下文预测 `[MASK]` 处的概率分布。  
4. **结果解码**：将预测的 Token ID 转换为中文词语，并返回可能性最高的候选结果。

---

### 实战示例
假设我们想补全句子：**"人工智能是未来的[MASK]。"**
```python
results = fill_mask("人工智能是未来的[MASK]。")
```

#### 输出结果（示例）：
```python
[
  {'score': 0.832, 'token': 4997, 'token_str': '趋势', 'sequence': '人工智能是未来的趋势。'},
  {'score': 0.102, 'token': 3423, 'token_str': '方向', 'sequence': '人工智能是未来的方向。'},
  {'score': 0.045, 'token': 2450, 'token_str': '核心', 'sequence': '人工智能是未来的核心。'},
  ...
]
```

#### 结果字段说明：
- `score`：预测结果的置信度（0~1，越高越可信）
- `token_str`：预测的中文词语
- `sequence`：补全后的完整句子

---

### 高级用法
1. **控制候选数量**：  
   ```python
   fill_mask("北京是中国的[MASK]。", top_k=5)  # 返回前5个候选结果
   ```
2. **处理多掩码**：  
   BERT 支持多个 `[MASK]`，但性能会随掩码数量增加而下降：
   ```python
   fill_mask("自然[MASK]处理是[MASK]能的核心技术。")
   ```
3. **自定义输入格式**：  
   通过 `tokenizer` 直接控制分词：
   ```python
   from transformers import AutoTokenizer, AutoModelForMaskedLM

   tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
   model = AutoModelForMaskedLM.from_pretrained("bert-base-chinese")
   inputs = tokenizer("机器学习需要[MASK]数据。", return_tensors="pt")
   outputs = model(**inputs)
   ```

---

### 注意事项
1. **掩码位置限制**：  
   BERT 对长文本中的掩码位置敏感，建议掩码靠近上下文中心以获得最佳效果。
2. **领域适配**：  
   BERT-Base-Chinese 训练语料以通用中文为主，专业领域（如医学、法律）需微调模型。
3. **性能优化**：  
   添加 `device=0` 参数启用 GPU 加速：
   ```python
   fill_mask = pipeline("fill-mask", model="bert-base-chinese", device=0)
   ```

通过这种方式，你可以快速实现中文文本的智能补全、文本纠错等应用场景。

In [15]:
from transformers import pipeline

fill_mask = pipeline(task="fill-mask", model="bert-base-chinese")

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
text = "人民是[MASK]可战胜的"

fill_mask(text, top_k=1)

[{'score': 0.9203723669052124,
  'token': 679,
  'token_str': '不',
  'sequence': '人 民 是 不 可 战 胜 的'}]

#### 设置文本补全的条数

In [17]:
text = "美国的首都是[MASK]"

fill_mask(text, top_k=1)

[{'score': 0.7596911191940308,
  'token': 8043,
  'token_str': '？',
  'sequence': '美 国 的 首 都 是 ？'}]

In [18]:
text = "巴黎是[MASK]国的首都。"
fill_mask(text, top_k=1)

[{'score': 0.9911912083625793,
  'token': 3791,
  'token_str': '法',
  'sequence': '巴 黎 是 法 国 的 首 都 。'}]

In [19]:
text = "美国的首都是[MASK]"
fill_mask(text, top_k=3)

[{'score': 0.7596911191940308,
  'token': 8043,
  'token_str': '？',
  'sequence': '美 国 的 首 都 是 ？'},
 {'score': 0.21126744151115417,
  'token': 511,
  'token_str': '。',
  'sequence': '美 国 的 首 都 是 。'},
 {'score': 0.026834219694137573,
  'token': 8013,
  'token_str': '！',
  'sequence': '美 国 的 首 都 是 ！'}]

In [21]:
text = "美国的首都是[MASK][MASK][MASK]"

fill_mask(text, top_k=1)

[[{'score': 0.5740261673927307,
   'token': 5294,
   'token_str': '纽',
   'sequence': '[CLS] 美 国 的 首 都 是 纽 [MASK] [MASK] [SEP]'}],
 [{'score': 0.4926738142967224,
   'token': 5276,
   'token_str': '约',
   'sequence': '[CLS] 美 国 的 首 都 是 [MASK] 约 [MASK] [SEP]'}],
 [{'score': 0.9353252053260803,
   'token': 511,
   'token_str': '。',
   'sequence': '[CLS] 美 国 的 首 都 是 [MASK] [MASK] 。 [SEP]'}]]

在BERT模型的填充掩码任务中，输入“美国的首都是[MASK][MASK][MASK]”得到“纽约。”而非“华盛顿”，这一现象可以从以下几个角度解释：

---

### 1. **训练数据与语言习惯的偏向**
   - **中文语料中的高频组合**：BERT-base-chinese是基于大规模中文语料（如新闻、百科）训练的。在中文语境中，“纽约”作为美国知名城市，其出现频率可能远高于“华盛顿”。例如，新闻中可能更多提及“纽约的金融中心”而非“华盛顿的政治职能”。
   - **地理知识的局限性**：预训练模型并非知识库，其预测依赖于文本中的统计规律而非真实世界知识。若训练数据中“华盛顿”与“首都”的关联不足，模型可能更倾向于选择高频词组合。

---

### 2. **分词与掩码机制的影响**
   - **基于字的分词策略**：BERT-base-chinese采用字级别的分词（如“纽”“约”单独处理），而非词级别（如“华盛顿”作为一个整体词）。当连续多个`[MASK]`存在时，模型会逐个预测每个位置的字符，而非整体生成连贯的词汇。例如：
     - 第一个`[MASK]`预测“纽”（概率最高），
     - 第二个`[MASK]`在“纽”后预测“约”，
     - 第三个`[MASK]`可能因上下文不完整而预测句号“。”。
   - **掩码独立预测的局限性**：模型对每个`[MASK]`的预测是独立的，无法协调多个掩码之间的语义连贯性，导致局部最优但整体错误的结果。

---

### 3. **模型架构与任务设计的限制**
   - **掩码语言模型（MLM）的局部性**：BERT的MLM任务在训练时通常仅遮盖单个词或少量词，而连续多个`[MASK]`的预测能力较弱。模型更擅长根据局部上下文预测单个词，但难以生成需多词组合的答案（如“华盛顿”需要三个字符的精确匹配）。
   - **预训练目标的偏差**：BERT的预训练任务（如“下一句预测”）侧重于句子间关系，而非事实性知识。即使模型知道“华盛顿是首都”，其生成也可能受概率分布影响而偏向高频词。

---

### 4. **解决方案与改进建议**
   - **使用知识增强型模型**：若需准确的地理知识，可尝试结合知识图谱或使用经过知识增强的模型（如ERNIE）。
   - **调整输入方式**：将问题拆分为单掩码预测（如“美国的首都是华[MASK][MASK]”），逐步引导模型生成正确结果。
   - **微调模型**：在包含“华盛顿-首都”关联的特定数据集上微调模型，强化其对该类知识的记忆。

---

### 总结
此现象反映了BERT模型的统计学习本质：其输出依赖于训练数据的分布和任务设计，而非真实世界的知识库。对于需要精确事实的问题，建议结合外部知识或使用专门优化的模型。

#### 思考：sequence 中出现的 [CLS] 和 [SEP] 是什么？

在BERT等基于Transformer的预训练模型中，`[CLS]`和`[SEP]`是两种特殊符号（Special Tokens），它们在模型输入和输出中扮演关键角色。以下是它们的核心作用解析：

---

### 一、`[CLS]`：**分类标记**（Classification Token）
- **位置**：始终位于输入序列的**起始位置**（第一个Token）。
- **核心功能**：
  1. **聚合全局信息**：在训练过程中，`[CLS]`位置的向量会被设计为捕获整个输入序列的语义信息。
  2. **分类任务输出**：在文本分类（如情感分析、文本匹配）任务中，模型会将`[CLS]`对应的最终隐藏层输出作为分类特征向量。例如：
     ```python
     # BERT用于二分类任务的输出层
     outputs = model(**inputs)
     logits = outputs.last_hidden_state[:, 0, :]  # 提取[CLS]位置的向量
     ```

- **可视化理解**：  
  ![](https://jalammar.github.io/images/bert-cls-token.png)  
  （图示：`[CLS]`向量通过全连接层映射到分类结果）

---

### 二、`[SEP]`：**分隔标记**（Separator Token）
- **位置**：用于分隔输入序列中的**不同部分**（如句子对、问答对）。
- **核心功能**：
  1. **句子对任务**：在问答、文本匹配等任务中分隔两个句子。例如输入格式：
     ```
     [CLS] 今天天气如何？ [SEP] 明天会下雨吗？ [SEP]
     ```
  2. **单句任务**：在单句输入中标记序列结束（可选，非必须）。

- **典型应用场景**：
  ```python
  # 句子对输入（如文本相似度计算）
  text_pair = ["我爱自然语言处理", "NLP技术非常有趣"]
  inputs = tokenizer(text_pair, return_tensors="pt", padding=True)
  # 编码结果：[CLS] 我 爱 自 然 语 言 处 理 [SEP] N L P 技 术 非 常 有 趣 [SEP]
  ```

---

### 三、实战中的注意事项
1. **自动添加机制**：  
   Hugging Face的`tokenizer`会自动添加`[CLS]`和`[SEP]`，开发者无需手动插入。例如：
   ```python
   inputs = tokenizer("你好，世界！", return_tensors="pt")
   print(inputs["input_ids"])  # 输出包含[CLS]和[SEP]
   ```

2. **序列长度限制**：  
   BERT的最大输入长度通常为512个Token（包括`[CLS]`和`[SEP]`），超长文本需要截断或分块处理。

3. **输出中的特殊符号处理**：  
   在使用`pipeline`时，后处理模块会**自动去除**`[CLS]`和`[SEP]`。例如：
   ```python
   fill_mask = pipeline("fill-mask", model="bert-base-chinese")
   result = fill_mask("人工智能是未来的[MASK]。")
   print(result[0]["sequence"])  # 输出："人工智能是未来的趋势。"（无特殊符号）
   ```

---

### 四、为什么用户会看到`[CLS]`和`[SEP]`？
在之前的代码示例中，用户直接打印了包含原始符号的序列，可能是因为：
1. **调试模式**：直接输出了未经后处理的模型原始输出（含特殊符号）。
2. **手动拼接结果**：未使用完整的Pipeline后处理流程，导致符号未被过滤。

---

### 五、总结
| 符号    | 作用场景                   | 开发者注意事项                |
|---------|----------------------------|-----------------------------|
| `[CLS]` | 分类、语义聚合任务         | 分类任务需使用此位置的向量   |
| `[SEP]` | 句子对任务、序列结束标记   | 处理多段文本时必须正确分隔   |

理解这些符号的作用，能帮助开发者更好地设计输入格式和解析模型输出，尤其是在微调自定义任务时。

## 使用 AutoClass 高效管理 `Tokenizer` 和 `Model`

通常，您想要使用的模型（网络架构）可以从您提供给 `from_pretrained()` 方法的预训练模型的名称或路径中推测出来。

AutoClasses就是为了帮助用户完成这个工作，以便根据`预训练权重/配置文件/词汇表的名称/路径自动检索相关模型`。

比如手动加载`bert-base-chinese`模型以及对应的 `tokenizer` 方法如下：

```python
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = AutoModel.from_pretrained("bert-base-chinese")
```

以下是我们实际操作和演示：

### 使用 `from_pretrained` 方法加载指定 Model 和 Tokenizer 

以下是关于使用 `AutoClass` 高效管理 Hugging Face Transformers 模型和分词器的详细解析，结合代码示例和实际应用场景：

---

### 一、AutoClass 的核心优势
**`AutoTokenizer` 和 `AutoModel` 是动态加载工具**，能够根据模型名称或路径自动匹配对应的类。其优势在于：
1. **无需手动指定类名**：例如加载 BERT 时无需显式使用 `BertTokenizer` 或 `BertModel`。
2. **跨模型兼容性**：同一套代码可无缝切换不同架构的模型（如 BERT → RoBERTa → ALBERT）。
3. **自动配置解析**：自动获取模型的配置文件（如 `config.json`），确保模型参数与架构匹配。

---

### 二、代码逐行解析
```python
from transformers import AutoTokenizer, AutoModel

# 指定模型名称（支持Hugging Face Hub的ID或本地路径）
model_name = "bert-base-chinese" 

# 加载分词器：自动匹配 BertTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 加载模型：自动匹配 BertModel
model = AutoModel.from_pretrained(model_name)
```

#### 关键方法 `from_pretrained` 的工作原理：
1. **模型识别**：根据 `model_name` 从 Hugging Face Hub 或本地缓存查找模型。
2. **配置文件加载**：读取 `config.json` 确定模型架构参数（如层数、注意力头数）。
3. **权重加载**：下载或加载本地的 `pytorch_model.bin` 文件初始化模型参数。
4. **分词器初始化**：加载对应的 `tokenizer.json` 或 `vocab.txt` 文件。

---

### 三、AutoModel 的输出结构
加载后的 `model` 接受分词器的输出，返回高维向量表示：
```python
inputs = tokenizer("自然语言处理", return_tensors="pt")
outputs = model(**inputs)

# outputs 包含：
# - last_hidden_state: 所有Token的隐藏状态 (shape: [batch_size, seq_len, hidden_dim])
# - pooler_output: [CLS]标记的池化表示 (shape: [batch_size, hidden_dim])
print(outputs.last_hidden_state.shape)  # 示例输出: torch.Size([1, 7, 768])
```

---

### 四、高级用法与技巧
#### 1. **设备管理与量化**
```python
# GPU加速（需CUDA环境）
model = AutoModel.from_pretrained(model_name, device_map="auto")

# 4-bit量化（减少显存占用）
from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModel.from_pretrained(model_name, quantization_config=bnb_config)
```

#### 2. **自定义模型路径**
```python
# 加载本地模型（需包含config.json和pytorch_model.bin）
local_path = "./models/bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(local_path)
model = AutoModel.from_pretrained(local_path)
```

#### 3. **任务专用模型**
对于特定任务，应使用对应的 `AutoModelForXXX` 类：
```python
from transformers import AutoModelForSequenceClassification

# 文本分类任务
cls_model = AutoModelForSequenceClassification.from_pretrained(model_name)
```

---

### 五、实际应用场景
| 场景                | 代码示例                                                                 |
|---------------------|--------------------------------------------------------------------------|
| **文本特征提取**     | 使用 `outputs.last_hidden_state` 作为下游任务的输入特征                  |
| **句子相似度计算**   | 比较 `pooler_output` 的余弦相似度                                       |
| **模型微调**         | 在 `AutoModel` 基础上添加自定义分类层                                    |
| **多模态任务**       | 结合 `AutoImageProcessor` 处理图像输入                                   |

---

### 六、注意事项
1. **网络依赖**：首次使用 `from_pretrained` 需联网下载模型（约500MB~数GB）。
2. **存储路径**：默认缓存路径为 `~/.cache/huggingface/hub`，可通过 `HF_HOME` 环境变量修改。
3. **版本兼容性**：确保 `transformers` 库版本与模型配置文件兼容（避免 `config.json` 解析错误）。

通过 `AutoClass`，开发者可以高效地在不同预训练模型间切换，快速实现从实验到部署的全流程。

### 通常，您想要使用的模型（网络架构）可以从您提供给 `from_pretrained()` 方法的预训练模型的名称或路径中推测出来。

这句话描述了 Hugging Face Transformers 库中 **AutoClass 的核心设计理念**：开发者只需提供预训练模型的名称或本地路径，库就能自动推断出对应的模型架构（如 BERT、GPT、RoBERTa 等），无需手动指定具体类名。以下是分步骤的通俗解释：

---

### 一、核心原理
1. **模型名称/路径** → 2. **解析配置文件** → 3. **匹配架构类**  
   - 当你调用 `AutoModel.from_pretrained("bert-base-chinese")` 时：
     - **步骤 1**：库会根据名称 `bert-base-chinese` 找到对应的配置文件（`config.json`）。
     - **步骤 2**：读取配置文件中的 `architectures` 字段（如 `"architectures": ["BertForMaskedLM"]`）。
     - **步骤 3**：自动选择 `BertForMaskedLM` 这个类来实例化模型。

---

### 二、关键机制
#### 1. **模型名称的隐含信息
   - **示例名称**：`bert-base-chinese`、`roberta-large`、`gpt2-xl`  
   - **命名规则**：模型名称通常包含架构标识（如 `bert`、`gpt2`）和配置参数（如 `base`、`large`）。
   - **自动映射表**：Transformers 库内部维护了一个映射表，将名称前缀（如 `bert-*`）映射到对应的模型类（`BertModel`）。

#### 2. **配置文件的桥梁作用
   每个预训练模型目录下都有一个 `config.json`，其中明确声明了模型架构：
   ```json
   {
     "model_type": "bert",        // 模型类型
     "architectures": ["BertLMHeadModel"],  // 具体类名
     "hidden_size": 768,
     ...
   }
   ```

---

### 三、实际场景示例
| 模型名称或路径                  | AutoClass 推断的模型类       | 应用场景              |
|---------------------------------|-----------------------------|---------------------|
| `"bert-base-chinese"`           | `BertModel`                 | 中文文本编码         |
| `"gpt2"`                        | `GPT2LMHeadModel`           | 英文文本生成         |
| `"./local_model/"`（本地目录）   | 根据本地 `config.json` 推断 | 自定义微调模型       |

---

### 四、为什么需要这种设计？
1. **简化代码**：同一套代码可加载不同架构的模型，无需手动修改类名。
   ```python
   # 只需改模型名，代码不变
   model = AutoModel.from_pretrained("roberta-base")  # 自动加载 RobertaModel
   ```
2. **兼容性保障**：确保模型架构与权重文件严格匹配（避免因手动选错类导致参数不匹配）。
3. **灵活扩展**：支持新模型架构只需更新映射表和配置文件，无需修改用户代码。

---

### 五、注意事项
- **自定义模型**：如果本地模型的 `config.json` 未正确设置 `model_type` 或 `architectures`，AutoClass 可能无法正确推断。此时需手动指定类：
  ```python
  from transformers import BertModel
  model = BertModel.from_pretrained("./custom_model/")
  ```
- **模型别名**：Hugging Face Hub 上的某些模型可能有别名（如 `"microsoft/deberta-v3-base"`），但最终仍通过配置文件推断真实架构。

通过这种设计，开发者可以更专注于任务本身，而非模型架构的底层细节。

In [22]:
from transformers import AutoTokenizer, AutoModel

model_name = "bert-base-chinese"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

#### 使用 BERT Tokenizer 编码文本

编码 (Encoding) 过程包含两个步骤：

- 分词：使用分词器按某种策略将文本切分为 tokens；
- 映射：将 tokens 转化为对应的 token IDs。

这段代码的运作逻辑如下：

---

### **代码逐行解析**
```python
model_name = "bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(model_name)  # 关键步骤
sequence = "美国的首都是华盛顿特区"
tokens = tokenizer.tokenize(sequence)  # 分词操作
```

---

### **关键过程详解**
1. **自动匹配 Tokenizer 类**  
   - `AutoTokenizer.from_pretrained(model_name)` 会根据 `model_name` 的值（如 `bert-base-chinese`）自动选择对应的分词器类（如 `BertTokenizer`）。
   - **底层逻辑**：
     - 检查 `model_name` 对应的配置文件（`tokenizer_config.json`）。
     - 根据配置文件中的 `"tokenizer_class": "BertTokenizer"` 字段，实例化 `BertTokenizer`。

2. **中文分词策略**  
   - **bert-base-chinese 的分词特性**：
     - 基于 **字级别（Character-level）** 的分词，每个汉字单独成词。
     - **例外**：部分常见词语可能合并（如“京”+“都” → “京都”），但大部分情况会拆分为单字。
   - **示例输出**：
     ```python
     print(tokens)  # 输出：['美', '国', '的', '首', '都', '是', '华', '盛', '顿', '特', '区']
     ```

3. **分词过程的技术细节**  
   - **步骤分解**：
     1. **文本规范化**：统一为 Unicode 编码，处理全角/半角符号。
     2. **预分词**：按空格和标点符号初步切分（对中文影响较小）。
     3. **子词切分**：根据预训练词汇表（`vocab.txt`）匹配最长的子词组合（对中文来说，通常直接按字切分）。

---

### **为什么选择 `AutoTokenizer` ？**
| 直接指定类（如 `BertTokenizer`）       | 使用 `AutoTokenizer`                |
|---------------------------------------|-------------------------------------|
| `tokenizer = BertTokenizer.from_pretrained(...)` | `tokenizer = AutoTokenizer.from_pretrained(...)` |
| 需提前知道模型架构                    | 自动适配任意模型（BERT、GPT、T5 等） |
| 代码与模型强耦合                      | 同一套代码可切换不同模型             |

---

### **高级应用场景**
1. **控制分词细节**：
   ```python
   # 禁止合并子词（强制按字切分）
   tokens = tokenizer.tokenize(sequence, never_split=["美", "国"])
   ```

2. **处理特殊符号**：
   - 自动添加模型所需的特殊标记（如 `[CLS]`、`[SEP]`），需调用 `encode` 方法：
     ```python
     encoded_input = tokenizer.encode(sequence)  # 输出：[101, 4997, 1744, ..., 102]
     ```

3. **批量处理与填充**：
   ```python
   inputs = tokenizer(
       [sequence1, sequence2], 
       padding=True, 
       truncation=True, 
       return_tensors="pt"
   )
   ```

---

### **常见问题排查**
1. **分词结果不符合预期**：
   - 检查词汇表文件 `vocab.txt`（路径：`tokenizer.vocab_file`）。
   - 尝试直接查看词汇表内容：`print(tokenizer.get_vocab())`。

2. **生僻字处理**：
   - 未登录词（OOV）会被标记为 `[UNK]`，可通过扩展词汇表解决。

3. **性能优化**：
   - 添加 `use_fast=True` 参数启用 Rust 加速的分词器（需模型支持）：
     ```python
     tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
     ```

---

通过 `AutoTokenizer`，开发者无需关心不同模型的分词器差异，只需指定模型名称即可高效完成文本预处理。这种设计显著提升了代码的可维护性和跨模型兼容性。

In [29]:
# 第一步：分词
sequence = "美国的首都是华盛顿特区"
tokens = tokenizer.tokenize(sequence)
print(tokens)

['美', '国', '的', '首', '都', '是', '华', '盛', '顿', '特', '区']


In [30]:
# 第二步：映射
token_ids = tokenizer.convert_tokens_to_ids(tokens)

In [34]:
print(token_ids)

[5401, 1744, 4638, 7674, 6963, 3221, 1290, 4670, 7561, 4294, 1277]


#### 使用 Tokenizer.encode 方法端到端处理


这段代码使用 `tokenizer.encode()` 方法将原始中文文本转换为模型可理解的数字序列（Token IDs），这是将文本输入模型的**核心预处理步骤**。具体过程如下：

---

### 一、代码解析与输出结果
```python
token_ids_e2e = tokenizer.encode("美国的首都是华盛顿特区")
print(token_ids_e2e)  # 输出: [101, 5401, 1744, ..., 102]
```
- **输入**：`"美国的首都是华盛顿特区"`
- **输出**：`[101, 5401, 1744, 4638, 7674, 6963, 3221, 1290, 4670, 7561, 4294, 1277, 102]`

---

### 二、`encode()` 方法的核心作用
`tokenizer.encode()` 方法将文本处理流程封装为三个关键步骤：
1. **分词（Tokenization）**  
   将文本拆分为模型能理解的子单元（Token）。对于 `bert-base-chinese`，默认按**字级别**切分：
   ```python
   tokens = tokenizer.tokenize("美国的首都是华盛顿特区")
   # 输出: ['[CLS]', '美', '国', '的', '首', '都', '是', '华', '盛', '顿', '特', '区', '[SEP]']
   ```

2. **添加特殊标记（Special Tokens）**  
   - `[CLS]`（ID 101）：序列起始标记，用于分类任务的特征聚合。
   - `[SEP]`（ID 102）：序列结束标记。

3. **转换为Token ID**  
   根据模型的词汇表（`vocab.txt`）将每个Token映射为数字：
   ```
   '美' → 5401  
   '国' → 1744  
   '的' → 4638  
   ...  
   '[SEP]' → 102
   ```

---

### 三、输出结果的详细拆解
| 位置 | Token ID | 对应文本 | 作用说明               |
|------|----------|----------|------------------------|
| 0    | 101      | `[CLS]`  | 序列开始，分类特征来源 |
| 1    | 5401     | 美       | 文本的第一个字         |
| 2    | 1744     | 国       |                        |
| 3    | 4638     | 的       |                        |
| 4    | 7674     | 首       |                        |
| 5    | 6963     | 都       |                        |
| 6    | 3221     | 是       |                        |
| 7    | 1290     | 华       |                        |
| 8    | 4670     | 盛       |                        |
| 9    | 7561     | 顿       |                        |
| 10   | 4294     | 特       |                        |
| 11   | 1277     | 区       | 文本的最后一个字       |
| 12   | 102      | `[SEP]`  | 序列结束               |

---

### 四、为什么需要这样做？
1. **模型输入标准化**：神经网络只能处理数字，因此必须将文本转换为数值向量。
2. **结构化输入**：特殊标记帮助模型理解输入结构（如句子边界）。
3. **与预训练一致**：确保输入格式与模型训练时的数据格式匹配，保证性能。

---

### 五、扩展应用：`encode()` vs `tokenize()`
| 方法                | 输出类型 | 包含特殊标记 | 典型用途               |
|---------------------|----------|--------------|------------------------|
| `tokenizer.tokenize()` | 字符串列表 | 可选         | 查看分词结果，调试分词 |
| `tokenizer.encode()`   | 整数列表   | 是           | 直接输入模型           |

- **手动控制特殊标记**：通过参数 `add_special_tokens=False` 可以禁用自动添加：
  ```python
  no_special_tokens = tokenizer.encode(sequence, add_special_tokens=False)
  # 输出: [5401, 1744, 4638, ..., 1277]
  ```

---

### 六、验证反向解码
可以通过 `decode()` 方法将 Token ID 还原为文本：
```python
decoded_text = tokenizer.decode(token_ids_e2e)
print(decoded_text)  # 输出: "[CLS] 美国的首都是华盛顿特区 [SEP]"
```
- **注意**：解码时会自动去除子词合并的标记（如 `##`），并还原特殊符号为可读形式。

---

通过这种方式，开发者可以高效地将任意文本转换为模型可处理的输入格式，这也是使用预训练模型进行推理或微调的第一步。

In [32]:
token_ids_e2e = tokenizer.encode(sequence)

In [33]:
token_ids_e2e

[101, 5401, 1744, 4638, 7674, 6963, 3221, 1290, 4670, 7561, 4294, 1277, 102]

这串数字确实是 **Token ID**，也就是模型内部用来表示每个文本单元的**唯一数字编码**。以下是更详细的解释：

---

### **Token ID 的本质**
- **定义**：每个 Token（如字、词或子词）在模型的**词汇表**（vocab）中对应的唯一编号。
- **作用**：将人类可读的文本转换为模型可理解的数字序列。
- **类比**：就像字典中的页码，每个单词对应一个特定编号。

---

### **结合你的代码示例**
```python
sequence = "美国的首都是华盛顿特区"
token_ids = tokenizer.encode(sequence)  # 输出: [101, 5401, 1744, ..., 102]
```

#### **每个 Token ID 的意义**
| Token ID | 对应内容          | 说明                 |
|----------|-------------------|----------------------|
| 101      | `[CLS]`          | 序列起始标记（自动添加） |
| 5401     | `美`             | 文本中的第一个汉字     |
| 1744     | `国`             |                      |
| 102      | `[SEP]`          | 序列结束标记（自动添加） |

---

### **为什么需要 Token ID？**
1. **模型本质需求**：神经网络只能处理数值，无法直接理解文本。
2. **标准化处理**：统一不同语言的字符编码（如中文、英文、符号）。
3. **高效计算**：数字索引比字符串操作更快速（GPU/TPU 擅长矩阵运算）。

---

### **验证 Token ID 的方法**
1. **查看词汇表**：  
   ```python
   # 获取词汇表（字典形式：{token: id}）
   vocab = tokenizer.get_vocab()
   print(vocab["美"])  # 输出: 5401
   ```

2. **反向解码**：  
   ```python
   # 将 Token ID 转回文本
   decoded_text = tokenizer.decode(token_ids)
   print(decoded_text)  # 输出: "[CLS] 美国的首都是华盛顿特区 [SEP]"
   ```

---

### **特殊情况的处理**
1. **未知词（OOV）**：  
   如果文本中的字不在词汇表中，会被标记为 `[UNK]`（ID 100）。例如：
   ```python
   tokenizer.encode("魑魅魍魉")  # 可能输出 [101, 100, 100, 100, 102]
   ```

2. **子词切分**：  
   对于英文模型（如 GPT-2），长词会被拆分为子词并分配多个 ID：
   ```python
   tokenizer = AutoTokenizer.from_pretrained("gpt2")
   tokenizer.encode("ChatGPT")  # 输出: [15496, 4281, 28265]
   # 对应子词: ["Chat", "G", "PT"]
   ```

---

### **总结**
- **Token ID 是模型与文本的桥梁**，贯穿整个 NLP 流程（输入→处理→输出）。
- **AutoTokenizer 自动完成**：从文本到 Token ID 的转换（包括分词、添加特殊标记）。
- **实践建议**：调试时多用 `tokenizer.tokenize()` 和 `tokenizer.decode()` 观察中间结果。

理解 Token ID 的机制，是掌握大模型微调和推理的关键一步！后续遇到生成结果异常时，也可以优先检查 Token ID 的转换是否正确。

#### 思考：为什么前后新增了 101 和 102？

在BERT等基于Transformer的预训练模型中，`101`（`[CLS]`）和`102`（`[SEP]`）是**特殊标记的Token ID**，它们的添加是模型输入格式的强制要求。以下是详细的解释：

---

### 一、`[CLS]`（ID 101）和`[SEP]`（ID 102）的作用
| 标记      | 作用场景               | 必要性               | 对应ID |
|-----------|------------------------|----------------------|--------|
| `[CLS]`   | **序列起始位置**       | 分类任务的特征来源   | 101    |
| `[SEP]`   | **序列结束位置**       | 分隔句子或标记边界   | 102    |

---

### 二、为什么必须添加这两个标记？
1. **预训练的一致性**  
   BERT在预训练阶段（如Masked Language Model和Next Sentence Prediction）就使用了`[CLS]`和`[SEP]`。模型通过大量数据学习到：
   - `[CLS]`的隐藏状态能聚合整个序列的语义信息（适合分类任务）。
   - `[SEP]`用于区分不同句子（即使单句输入也需要标记边界）。

2. **模型架构的硬性要求**  
   BERT的输入层（Embedding Layer）和注意力机制（Self-Attention）在设计时已固定了以下规则：
   ```python
   # 输入格式的强制要求
   input_ids = [CLS] + token_ids + [SEP]
   ```

3. **任务适配性**  
   - **单句任务**（如文本分类）：`[CLS]`的输出用于分类。
   - **句子对任务**（如问答）：`[SEP]`分隔两个句子：
     ```
     [CLS] 句子1 [SEP] 句子2 [SEP]
     ```

---

### 三、实例验证
#### 1. **查看特殊标记的映射**
```python
print(tokenizer.cls_token_id)  # 输出: 101
print(tokenizer.sep_token_id)  # 输出: 102
```

#### 2. **手动控制标记添加**
通过参数`add_special_tokens=False`可以禁用自动添加：
```python
raw_ids = tokenizer.encode("美国的首都是华盛顿特区", add_special_tokens=False)
print(raw_ids)  # 输出: [5401, 1744, 4638, ..., 1277]（无101和102）
```

#### 3. **添加标记后的输入结构**
```python
# 原始输入（未分词）
text = "美国的首都是华盛顿特区"

# 模型实际接收的输入
model_input = "[CLS] 美 国 的 首 都 是 华 盛 顿 特 区 [SEP]"
```

---

### 四、其他模型的特殊标记对比
| 模型类型       | 起始标记       | 结束标记       | 分隔标记       |
|----------------|----------------|----------------|----------------|
| BERT           | `[CLS]` (101)  | `[SEP]` (102)  | `[SEP]` (102)  |
| GPT-2          | `<|endoftext|>`| `<|endoftext|>`| 无             |
| RoBERTa        | `<s>` (0)      | `</s>` (2)     | `</s>` (2)     |
| T5             | `</s>` (1)     | `</s>` (1)     | 无             |

---

### 五、总结：为什么必须保留这些标记？
1. **模型性能保障**：移除会导致输入分布与预训练不一致，显著降低效果。
2. **任务兼容性**：分类、问答、文本生成等任务依赖这些标记的结构化输入。
3. **注意力机制优化**：`[CLS]`的位置通常被设计为全局注意力汇聚点。

因此，除非特殊需求（如模型蒸馏或自定义训练），否则应保持默认的标记添加逻辑。

## 是否所哟模型都使用 `[CLS]` 和 `[SEP]`？

并非所有模型都使用 `[CLS]` 和 `[SEP]`。**特殊标记的存在取决于模型的架构设计和预训练任务**。以下是不同模型类别的对比分析：

---

### 一、不同模型对特殊标记的需求
| 模型类型       | 典型代表      | 特殊标记                                                                 | 作用场景               |
|----------------|-------------|--------------------------------------------------------------------------|------------------------|
| **自编码模型**   | BERT        | `[CLS]`（起始）、`[SEP]`（分隔/结束）                                       | 句子分类、掩码预测       |
| **自回归模型**   | GPT系列      | `<\|endoftext\|>`（结束标记）                                               | 文本生成、续写任务       |
| **序列到序列**   | T5、BART     | `</s>`（结束标记，兼具分隔作用）                                             | 翻译、摘要等生成任务     |
| **多模态模型**   | CLIP        | 无特殊标记（直接拼接文本和图像特征）                                         | 图文匹配、跨模态理解     |
| **多语言模型**   | XLM-R       | `</s>`（结束标记）、`<s>`（起始标记）                                        | 跨语言文本理解           |

---

### 二、具体模型对比
#### 1. **BERT 及其变种**（RoBERTa、ALBERT）
- **必须包含**：`[CLS]` 和 `[SEP]`  
- **输入格式**：  
  ```python
  # 单句输入
  [CLS] 今天天气真好 [SEP]
  
  # 句子对输入
  [CLS] 句子1 [SEP] 句子2 [SEP]
  ```

#### 2. **GPT 系列**（GPT-2、GPT-3）
- **无 `[CLS]`**，使用 `<|endoftext|>` 标记结束  
- **输入格式**：  
  ```python
  # 单轮对话
  "你好，有什么可以帮助你的吗？<|endoftext|>"
  
  # 多轮对话（部分定制模型）
  "用户：你好\n助手：<|endoftext|>"
  ```

#### 3. **T5 模型**
- **统一使用 `</s>`** 作为分隔符和结束符  
- **输入格式**：  
  ```python
  # 翻译任务（英→中）
  "translate English to Chinese: Hello world! </s>"
  
  # 摘要任务
  "summarize: 这是一篇长文本... </s>"
  ```

---

### 三、为什么模型之间存在差异？
1. **预训练目标的差异**：
   - BERT 的 MLM 任务需要 `[CLS]` 聚合全局信息，而 GPT 的生成任务只需结束符。
2. **架构设计的区别**：
   - 自编码模型（BERT）需要结构化输入，而自回归模型（GPT）更关注序列生成。
3. **多任务统一性**：
   - T5 将所有任务转换为文本到文本格式，因此统一使用 `</s>`。

---

### 四、实践建议
1. **查阅模型文档**：  
   在 Hugging Face 模型页面的 **Model Card** 中，查看 `tokenizer_config.json` 或 `config.json`，明确支持的标记。
   ```json
   // config.json 示例（BERT）
   {
     "model_type": "bert",
     "pad_token_id": 0,
     "sep_token_id": 102,
     "cls_token_id": 101
   }
   ```

2. **代码验证**：  
   通过 `tokenizer.special_tokens_map` 查看实际支持的标记：
   ```python
   from transformers import AutoTokenizer
   tokenizer = AutoTokenizer.from_pretrained("gpt2")
   print(tokenizer.special_tokens_map)
   # 输出: {'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', ...}
   ```

3. **自定义处理**：  
   若需去除特殊标记（如生成任务中），可通过参数控制：
   ```python
   inputs = tokenizer("文本", add_special_tokens=False)
   ```

---

### 五、总结
- **模型特殊性**：每个模型的特殊标记是其架构和训练目标的体现，不可随意混用。
- **灵活适配**：使用 `AutoTokenizer` 自动加载对应标记，无需硬编码。
- **调试技巧**：通过 `tokenizer.decode()` 反向验证输入是否符合预期格式。

理解这些差异，能帮助你在切换模型时快速调整输入处理逻辑，避免因格式错误导致的性能下降或运行失败。

In [39]:
tokenizer.decode(token_ids)

'美 国 的 首 都 是 华 盛 顿 特 区'

In [40]:
tokenizer.decode(token_ids_e2e)

'[CLS] 美 国 的 首 都 是 华 盛 顿 特 区 [SEP]'

#### 编解码多段文本

In [41]:
sequence_batch = ["美国的首都是华盛顿特区", "中国的首都是北京"]

In [42]:
token_ids_batch = tokenizer.encode(sequence_batch)

In [43]:
tokenizer.decode(token_ids_batch)

'[CLS] 美 国 的 首 都 是 华 盛 顿 特 区 [SEP] 中 国 的 首 都 是 北 京 [SEP]'

![](docs/images/bert_pretrain.png)

### 实操建议：直接使用 tokenizer.\_\_call\_\_ 方法完成文本编码 + 特殊编码补全

编码后返回结果：

```json
input_ids: token_ids
token_type_ids: token_id 归属的句子编号
attention_mask: 指示哪些token需要被关注（注意力机制）
```

代码和输出清晰地展示了如何使用 `tokenizer.__call__` 方法处理文本对（两个句子），并解析了返回的核心字段。以下是对输出结果的详细解释和优化建议：

---

### **一、输出字段解析**
#### 1. **`input_ids`**
- **作用**：将文本转换为模型可理解的 Token ID 序列，包含特殊标记。
- **结构**：
  ```python
  [
    101,        # [CLS] 标记（序列开始）
    ...         # 第一个句子的 Token ID（"美国的首都是华盛顿特区"）
    102,        # [SEP] 标记（分隔第一个句子）
    704, 1744,  # 第二个句子的 Token ID（"中国的首都是北京"）
    102         # [SEP] 标记（序列结束）
  ]
  ```
- **验证**：通过 `tokenizer.decode()` 可还原原始文本：
  ```python
  print(tokenizer.decode(input_ids))  
  # 输出: [CLS] 美国的首都是华盛顿特区 [SEP] 中国的首都是北京 [SEP]
  ```

---

#### 2. **`token_type_ids`**
- **作用**：标识每个 Token 属于哪个句子（用于句子对任务）。
- **编码规则**：
  - `0`：第一个句子的 Token（"美国的首都是华盛顿特区"）。
  - `1`：第二个句子的 Token（"中国的首都是北京"）。
- **示例**：
  ```python
  token_type_ids = [0, 0, ..., 0, 0, 1, 1, ..., 1, 1]
  ```

---

#### 3. **`attention_mask`**
- **作用**：指示哪些 Token 需要被模型关注（1 表示有效 Token，0 表示填充 Token）。
- **当前输出**：全为 `1`，表示没有填充（所有 Token 均为有效输入）。
- **填充场景示例**：若序列长度超过模型限制，会自动截断或填充：
  ```python
  inputs = tokenizer(text_pair, padding="max_length", max_length=512)
  # attention_mask 中会出现 0 表示填充部分
  ```

---

### **二、优化建议**
#### 1. **增强可读性**
为更直观理解输出，可以将 Token ID 映射回文本片段：
```python
def debug_tokenizer_output(output):
    decoded = tokenizer.decode(output["input_ids"])
    print(f"Decoded text: {decoded}\n")
    
    for key, value in output.items():
        if key == "input_ids":
            tokens = [tokenizer.decode([id]) for id in value]
            print(f"{key}: {tokens}")
        else:
            print(f"{key}: {value}")
        print("-" * 50)

debug_tokenizer_output(embedding_batch)
```

**输出示例**：
```
Decoded text: [CLS] 美国的首都是华盛顿特区 [SEP] 中国的首都是北京 [SEP]

input_ids: ['[CLS]', '美', '国', '的', '首', '都', '是', '华', '盛', '顿', '特', '区', '[SEP]', '中', '国', '的', '首', '都', '是', '北', '京', '[SEP]']
--------------------------------------------------
token_type_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
--------------------------------------------------
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
--------------------------------------------------
```

---

#### 2. **处理批量和长文本**
- **批量输入**：直接传入列表，自动处理填充：
  ```python
  texts = [
      "美国的首都是华盛顿特区",
      "中国的首都是北京",
      "日本的首都是东京"
  ]
  batch = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
  ```

- **设备优化**：生成张量并移至 GPU：
  ```python
  batch = {k: v.to("cuda") for k, v in batch.items()}
  ```

---

### **三、核心参数说明**
| 参数名            | 作用                           | 常用值                          |
|--------------------|--------------------------------|---------------------------------|
| `padding`          | 填充策略                       | `True`/`"longest"`/`"max_length"` |
| `truncation`       | 截断策略                       | `True`/`"only_first"`/`"only_second"` |
| `max_length`       | 最大序列长度                   | 512（BERT 默认）                |
| `return_tensors`   | 返回张量格式                   | `"pt"`（PyTorch） / `"tf"`（TensorFlow） |

---

### **四、实践场景**
1. **文本相似度计算**：  
   输入句子对，通过 `token_type_ids` 区分两个句子，计算语义相似度。
2. **问答任务**：  
   将问题和上下文拼接为 `[CLS] question [SEP] context [SEP]`，模型预测答案位置。
3. **生成任务**：  
   禁用 `token_type_ids` 和 `attention_mask` 的部分功能，适配生成模型。

通过合理配置 `tokenizer` 参数，可以适配从简单分类到复杂生成的全场景需求。

## tokenizer.__call__ 是什么

在 Hugging Face Transformers 库中，`tokenizer.__call__` 是 `PreTrainedTokenizer` 类的一个**核心方法**，它提供了一种简洁且功能全面的方式将原始文本转换为模型所需的输入格式。以下是其核心原理和使用场景的详细解释：

---

### **一、`__call__` 方法的本质**
1. **Python 的特殊方法**  
   `__call__` 是 Python 的魔术方法（Magic Method），允许类的实例像函数一样被直接调用。例如：
   ```python
   class MyTokenizer:
       def __call__(self, text):
           return self.encode(text)

   tokenizer = MyTokenizer()
   output = tokenizer("Hello world")  # 等价于 tokenizer.__call__("Hello world")
   ```

2. **Transformers 中的实现**  
   在 Transformers 库中，`tokenizer.__call__()` 封装了以下完整流程：
   ```python
   def __call__(self, text, ...):
       tokens = self.tokenize(text)          # 分词
       ids = self.convert_tokens_to_ids(tokens)  # 转 Token ID
       inputs = self.prepare_for_model(ids)  # 添加特殊标记、填充等
       return inputs
   ```

---

### **二、`__call__` 的核心功能**
当您直接调用 `tokenizer(text)` 时，它会自动完成以下处理：
1. **分词（Tokenization）**  
   将文本拆分为模型可理解的子单元（如字、词或子词）。
2. **添加特殊标记**  
   - 自动插入 `[CLS]`（起始）、`[SEP]`（结束/分隔）等标记。
3. **转换为数字格式**  
   - 生成 `input_ids`（Token ID 序列）
   - 生成 `token_type_ids`（句子归属标记，用于区分句子对）
   - 生成 `attention_mask`（标识有效 Token 位置）
4. **填充与截断**  
   根据参数自动处理序列长度（如 `padding=True` 时填充到相同长度）。

---

### **三、对比其他方法**
| 方法                | 输入类型       | 输出内容                     | 典型用途               |
|---------------------|---------------|-----------------------------|------------------------|
| `tokenizer.tokenize()` | 单条文本       | 分词后的字符串列表           | 查看分词结果           |
| `tokenizer.encode()`   | 单条文本       | 包含特殊标记的 Token ID 列表 | 简单编码               |
| `tokenizer.__call__()` | 单条/批量文本  | 结构化的字典（含所有编码信息）| 直接输入模型训练/推理  |

---

### **四、使用场景与示例**
#### 1. **基本用法（单条文本）**
```python
text = "中国的首都是北京"
inputs = tokenizer(text)  # 等价于 tokenizer.__call__(text)

print(inputs)
# 输出:
# {
#   'input_ids': [101, 704, 1744, 4638, 7674, 6963, 3221, 1266, 776, 102],
#   'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#   'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
# }
```

#### 2. **处理文本对（如问答、句子相似度）**
```python
text_pair = ("中国的首都是哪里？", "北京是中国的首都。")
inputs = tokenizer(*text_pair)

print(inputs["token_type_ids"])  
# 输出: [0,0,...0, 1,1,...1] （标识两个句子的归属）
```

#### 3. **批量处理与填充**
```python
texts = ["你好", "自然语言处理很有趣"]
batch = tokenizer(
    texts,
    padding=True,        # 自动填充到最长序列
    truncation=True,     # 超长时截断
    return_tensors="pt"  # 返回 PyTorch 张量
)
# 输出张量的形状:
# input_ids: torch.Size([2, max_length])
```

---

### **五、关键参数说明**
| 参数名              | 作用                                                                 | 示例值                 |
|---------------------|----------------------------------------------------------------------|------------------------|
| `padding`           | 填充策略（`True`/`"longest"`/`"max_length"`）                        | `padding="max_length"` |
| `truncation`        | 截断策略（`True`/`"only_first"`/`"only_second"`）                    | `truncation=True`      |
| `max_length`        | 最大序列长度                                                         | `max_length=512`       |
| `return_tensors`    | 返回的格式（`"pt"`/`"tf"`/`"np"`）                                    | `return_tensors="pt"`  |
| `add_special_tokens`| 是否添加特殊标记（如 `[CLS]`、`[SEP]`）                               | `add_special_tokens=False` |

---

### **六、为什么推荐使用 `__call__` 方法？**
1. **功能全面**：一次性完成分词、编码、填充、截断等所有预处理步骤。
2. **兼容性**：支持单条文本、文本对、批量输入，适配多种任务。
3. **生产就绪**：返回的字典可直接输入模型（无需手动组装张量）。

---

### **七、底层实现解析**
`tokenizer.__call__` 本质上是调用了 `tokenizer.encode_plus()` 方法，后者提供了更细粒度的控制。若需要高级功能（如返回字符偏移量），可直接使用：
```python
inputs = tokenizer.encode_plus(
    text,
    return_offsets_mapping=True  # 返回每个 Token 在原始文本中的位置
)
```

通过掌握 `tokenizer.__call__`，您可以高效地将任意文本转换为模型所需的输入格式，这是使用 Transformers 库进行 NLP 任务的关键技能。

##  token_type_ids 和 attention_mask是什么？

在 Hugging Face Transformers 库中，`token_type_ids` 和 `attention_mask` 是模型输入的两个关键字段，它们分别承担不同的作用。以下是对这两个字段的详细解释：

---

### **一、`token_type_ids`（句子归属标记）**
#### **作用**
- 用于标识输入中的不同部分（如句子对任务中的两个句子），帮助模型区分输入中的多个文本段。
- 在需要处理多个独立文本段的任务中（如问答、文本匹配），明确每个 Token 属于哪个句子。

#### **编码规则**
- **单句输入**：所有 Token 的 `token_type_ids` 均为 `0`。
- **句子对输入**：
  - 第一个句子的所有 Token 标记为 `0`。
  - 第二个句子的所有 Token 标记为 `1`。
  - 分隔符 `[SEP]` 的标记与其后句子的类型一致。

#### **示例**
```python
text_pair = ("中国的首都是北京", "北京是中国的首都")
inputs = tokenizer(*text_pair)

# 输出 token_type_ids
# [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
```
- **结构解析**：
  ```
  [CLS] 中 国 的 首 都 是 北 京 [SEP] 北 京 是 中 国 的 首 都 [SEP]
    0   0  0  0  0  0  0  0  0   0   1  1  1  1  1  1  1  1  1   1
  ```

#### **适用场景**
- 句子对分类（如文本相似度计算）。
- 问答任务（问题 + 上下文）。
- 多轮对话中的历史与当前输入。

#### **注意事项**
- 某些模型（如 RoBERTa）不依赖 `token_type_ids`，仅通过 `[SEP]` 分隔句子。
- 可通过参数 `return_token_type_ids=False` 禁用生成该字段。

---

### **二、`attention_mask`（注意力掩码）**
#### **作用**
- 标识哪些 Token 是实际输入内容（值为 `1`），哪些是填充部分（值为 `0`）。
- 在注意力机制中，模型会忽略 `attention_mask=0` 的位置，避免填充部分干扰计算。

#### **编码规则**
- 所有实际输入的 Token 标记为 `1`。
- 填充部分（Padding）的 Token 标记为 `0`。

#### **示例**
```python
texts = ["你好", "自然语言处理很有趣"]
inputs = tokenizer(texts, padding=True)

# 输出 attention_mask
# [
#   [1, 1, 1, 0, 0, 0, 0, 0],  # 第一条文本（短，填充到长度8）
#   [1, 1, 1, 1, 1, 1, 1, 1]   # 第二条文本（无填充）
# ]
```

#### **适用场景**
- 批量处理时统一序列长度（填充短文本）。
- 动态截断超长输入（如 `max_length=512`）。

#### **注意事项**
- 如果未启用填充（`padding=False`），所有位置的 `attention_mask` 均为 `1`。
- 在生成任务中，`attention_mask` 也用于控制解码时的可见范围（如仅关注上文）。

---

### **三、字段协作示例**
#### **输入文本对**
```python
text_pair = ("中国的首都是北京", "北京是中国的首都")
inputs = tokenizer(*text_pair, padding="max_length", max_length=16)
```

#### **输出结构**
| 字段名             | 值                                                                 |
|-------------------|-------------------------------------------------------------------|
| `input_ids`       | `[101, 704, 1744, ..., 102, 1266, 776, 102, 0, 0, 0, 0, 0]`       |
| `token_type_ids`  | `[0, 0, 0, ..., 0, 1, 1, 1, 0, 0, 0, 0, 0]`                       |
| `attention_mask`  | `[1, 1, 1, ..., 1, 1, 1, 1, 0, 0, 0, 0, 0]`                       |

- **解析**：
  - `input_ids` 包含实际 Token ID 和填充部分的 `0`。
  - `token_type_ids` 标识第二个句子，填充部分也标记为 `0`。
  - `attention_mask` 标识填充部分为 `0`。

---

### **四、总结**
| 字段名            | 核心作用                          | 关键场景                  | 默认行为                |
|------------------|----------------------------------|-------------------------|------------------------|
| `token_type_ids` | 区分多个文本段                   | 句子对任务、问答、多轮对话 | 自动生成（若模型需要）  |
| `attention_mask` | 标识有效 Token 和填充位置         | 批量处理、动态截断        | 自动填充（若启用填充）  |

理解这两个字段的用途，能帮助你：
1. **正确构造模型输入**：尤其是在处理复杂任务（如问答、对话）时。
2. **优化显存和计算**：通过合理设置填充策略减少无效计算。
3. **调试输入异常**：当模型输出异常时，检查掩码和类型标记是否匹配预期。

## 对于token_type_ids，第三个句子标记就是2？

在绝大多数基于 BERT 架构的模型中，**`token_type_ids` 仅支持 0 和 1 两种类型**，无法直接标记第三个句子（如 2）。这是由模型的设计和预训练任务决定的。以下是详细解释和解决方案：

---

### **一、标准模型的限制**
#### **BERT 系列（包括 RoBERTa、ALBERT 等）**
- **输入结构**：仅支持 **两个句子**（单句或句子对）。
- **token_type_ids 规则**：
  - `0`：第一个句子（或单句的所有部分）。
  - `1`：第二个句子。
  - **无法扩展**：模型权重和注意力机制在预训练时未接触过 `2`，强行使用会导致不可预测的行为。

#### **示例：输入两个句子**
```python
text_pair = ("句子1", "句子2")
inputs = tokenizer(*text_pair)
print(inputs.token_type_ids)
# 输出: [0, 0, ..., 0, 1, 1, ..., 1]
```

---

### **二、为什么不能标记为 2？**
1. **预训练任务的限制**  
   BERT 的预训练任务（如 Masked Language Model 和 Next Sentence Prediction）仅涉及两个句子，模型从未学习过 `token_type_ids=2` 的语义。

2. **模型架构的硬编码**  
   BERT 的 Token Type Embeddings 层（用于编码 `token_type_ids`）的维度固定为 2。若强行输入 `2`，会导致索引越界错误：
   ```python
   # 假设 token_type_embeddings 的维度为 2
   embeddings = token_type_embeddings(torch.tensor([2]))  # 报错：index out of range
   ```

3. **注意力机制的优化**  
   模型的自注意力机制在预训练时针对两种句子类型进行了优化，增加第三种类型会破坏权重分布。

---

### **三、多句子输入的解决方案**
#### **场景需求**  
需要处理三个或更多独立文本段（如：问题 + 上下文 + 补充说明）。

#### **解决方案**
1. **合并句子**  
   将部分句子合并为同一段，用 `[SEP]` 分隔，但共用 `token_type_id`：
   ```python
   # 输入结构（三个句子合并为两段）
   [CLS] 句子A [SEP] 句子B [SEP] 句子C [SEP]
   token_type_ids = [0, 0, ..., 0, 1, 1, ..., 1, 1]
   ```

2. **自定义模型扩展**  
   修改模型架构以支持更多 `token_type_ids`（需重新预训练）：
   ```python
   # 修改 BERT 配置（例如将 type_vocab_size 从 2 改为 3）
   config = BertConfig(type_vocab_size=3)
   model = BertModel(config)
   ```

3. **使用支持多段输入的模型**  
   部分模型（如 **MPNet**、**Longformer**）原生支持更灵活的分段标记，但需要查阅具体文档。

---

### **四、实战验证**
#### **尝试输入三个句子（错误示例）**
```python
texts = ("句子1", "句子2", "句子3")
inputs = tokenizer(*texts)  # 报错：只能输入 1 或 2 个文本段
```

#### **正确做法（合并为两段）**
```python
text_pair = ("句子1 和 句子2", "句子3")  # 将前两个句子合并
inputs = tokenizer(*text_pair)
```

---

### **五、总结**
- **标准限制**：BERT 系模型的 `token_type_ids` 仅支持 `0` 和 `1`，无法标记第三段。
- **替代方案**：合并文本段或选择支持多段的模型。
- **扩展建议**：若需处理复杂分段任务，推荐使用 **DeBERTa-v3** 或 **MPNet** 等更灵活的架构。

如果需要处理多个独立段落（如法律文档中的多个条款），建议改用 **标记段落边界符**（如自定义 `[PAR]`）并仅用 `0` 作为 `token_type_id`，或使用 **文档级模型**（如 DocBERT）。

## attention_mask是可以人工标注？

是的，**`attention_mask` 可以人工标注**，但需要遵循模型对输入格式的要求。以下是详细的解释和操作方法：

---

### **一、`attention_mask` 的作用原理**
- **核心功能**：标识哪些位置是有效输入（`1`），哪些是填充部分（`0`）。
- **模型行为**：在计算注意力权重时，模型会忽略 `attention_mask=0` 的位置。
- **自动生成**：当使用 `tokenizer` 时，`padding=True` 参数会自动生成 `attention_mask`。

---

### **二、手动标注的场景**
1. **自定义填充策略**  
   当输入数据已预先处理（如手动填充到固定长度），需自行生成掩码。

2. **部分注意力控制**  
   需要强制模型忽略某些 Token（如噪声数据、敏感信息）。

3. **高级任务需求**  
   如动态掩码（Dynamic Masking）、对抗训练等。

---

### **三、手动标注方法**
#### **示例 1：基础手动标注**
```python
import torch

# 假设 input_ids 是已填充的序列
input_ids = torch.tensor([
    [101, 2345, 3456, 102, 0, 0],   # 有效长度 4
    [101, 5678, 102, 0, 0, 0]      # 有效长度 3
])

# 手动创建 attention_mask
attention_mask = (input_ids != 0).int()

print(attention_mask)
# 输出:
# tensor([[1, 1, 1, 1, 0, 0],
#         [1, 1, 1, 0, 0, 0]], dtype=torch.int32)
```

#### **示例 2：部分注意力控制**
```python
# 强制模型忽略第一个句子的后半部分
input_ids = [101, 2345, 3456, 102, 5678, 6789, 102]
attention_mask = [1, 1, 1, 1, 0, 0, 0]  # 仅关注前 4 个 Token

inputs = {
    "input_ids": torch.tensor([input_ids]),
    "attention_mask": torch.tensor([attention_mask])
}

outputs = model(**inputs)
```

---

### **四、注意事项**
1. **格式一致性**  
   `attention_mask` 必须与 `input_ids` **形状相同**，且数据类型为整数（`int`）或布尔（`bool`）。

2. **模型兼容性**  
   所有基于 Transformer 的模型（如 BERT、GPT）都依赖 `attention_mask`，不可省略。

3. **特殊标记处理**  
   手动标注时需保留 `[CLS]` 和 `[SEP]` 的掩码为 `1`，否则可能破坏模型逻辑。

4. **性能影响**  
   错误的掩码可能导致注意力计算异常（如全部为 `0` 的序列会引发错误）。

---

### **五、高级应用：动态掩码**
在训练中随机屏蔽部分 Token（类似 BERT 的 MLM 任务）：
```python
import numpy as np

def dynamic_masking(input_ids, mask_prob=0.15):
    mask = np.random.rand(*input_ids.shape) < mask_prob
    # 保留 [CLS] 和 [SEP]
    mask[:, 0] = False
    mask[input_ids == 102] = False
    attention_mask = (~mask).astype(int)
    return torch.tensor(attention_mask)

# 使用示例
input_ids = torch.tensor([[101, 2345, 3456, 102, 0, 0]])
attention_mask = dynamic_masking(input_ids.numpy())
```

---

### **六、总结**
| 操作类型       | 推荐方式                     | 适用场景                   |
|---------------|----------------------------|--------------------------|
| **常规处理**   | 依赖 `tokenizer` 自动生成     | 大多数 NLP 任务            |
| **手动标注**   | 自定义逻辑生成               | 特殊需求或高级任务          |
| **动态调整**   | 在训练循环中实时生成         | 预训练、数据增强           |

通过合理使用 `attention_mask`，可以精细控制模型对输入的处理方式，但需确保符合模型架构的预期行为。

In [48]:
embedding_batch = tokenizer("美国的首都是华盛顿特区", "中国的首都是北京")
print(embedding_batch)

{'input_ids': [101, 5401, 1744, 4638, 7674, 6963, 3221, 1290, 4670, 7561, 4294, 1277, 102, 704, 1744, 4638, 7674, 6963, 3221, 1266, 776, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [49]:
# 优化下输出结构
for key, value in embedding_batch.items():
    print(f"{key}: {value}\n")

input_ids: [101, 5401, 1744, 4638, 7674, 6963, 3221, 1290, 4670, 7561, 4294, 1277, 102, 704, 1744, 4638, 7674, 6963, 3221, 1266, 776, 102]

token_type_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]



### 添加新 Token

当出现了词表或嵌入空间中不存在的新Token，需要使用 Tokenizer 将其添加到词表中。 Transformers 库提供了两种不同方法：

- add_tokens: 添加常规的正文文本 Token，以追加（append）的方式添加到词表末尾。
- add_special_tokens: 添加特殊用途的 Token，优先在已有特殊词表中选择（`bos_token, eos_token, unk_token, sep_token, pad_token, cls_token, mask_token`）。如果预定义均不满足，则都添加到`additional_special_tokens`。

#### 添加常规 Token

先查看已有词表，确保新添加的 Token 不在词表中：

在 Hugging Face Transformers 库中为分词器（Tokenizer）添加新 Token 是一个常见的需求，尤其是在处理领域特定术语、新词汇或特殊符号时。以下是关于该过程的详细说明和注意事项：

---

### **一、添加新 Token 的两种方法**
#### 1. **`add_tokens()`：添加常规文本 Token**
   - **特点**：
     - 将新 Token **追加到词表末尾**，分配新 ID。
     - 适用于普通词汇（如“天干”、“地支”）。
   - **步骤**：
     ```python
     from transformers import AutoTokenizer

     # 加载分词器
     tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

     # 检查原始词表大小
     print(f"Original vocab size: {len(tokenizer)}")  # 输出: 21128

     # 定义新 Token 并过滤已存在的
     new_tokens = ["天干", "地支"]
     new_tokens = list(set(new_tokens) - set(tokenizer.vocab.keys()))

     # 添加新 Token
     tokenizer.add_tokens(new_tokens)
     print(f"New vocab size: {len(tokenizer)}")  # 输出: 21130
     ```

#### 2. **`add_special_tokens()`：添加特殊 Token**
   - **特点**：
     - 优先使用预定义的保留字段（如 `[CLS]`、`[SEP]`）。
     - 若预定义字段不足，则添加到 `additional_special_tokens`。
   - **示例**：
     ```python
     # 添加特殊 Token（如领域分隔符）
     special_tokens = {"additional_special_tokens": ["[医学]", "[法律]"]}
     tokenizer.add_special_tokens(special_tokens)
     ```

---

### **二、关键注意事项**
#### 1. **调整模型嵌入层**
   **添加新 Token 后必须扩展模型的词嵌入矩阵**，否则模型无法识别新 Token：
   ```python
   from transformers import AutoModel

   model = AutoModel.from_pretrained("bert-base-chinese")
   model.resize_token_embeddings(len(tokenizer))  # 同步调整嵌入层
   ```

   - **原理**：模型初始化的嵌入矩阵大小与原始词表一致，新 Token 的嵌入需要初始化（默认随机）。

#### 2. **验证 Token 添加成功**
   - **检查分词结果**：
     ```python
     text = "天干地支是中国古代历法的基础。"
     tokens = tokenizer.tokenize(text)
     print(tokens)  # 期望输出包含 "天干" 和 "地支"
     ```
   - **查看 Token ID**：
     ```python
     print(tokenizer.convert_tokens_to_ids("天干"))  # 输出新 ID（如 21128）
     ```

#### 3. **处理子词冲突**
   - **问题**：若新 Token 可被拆分为现有子词（如“天干” → “天” + “干”），需确保其作为独立 Token 优先匹配。
   - **解决方案**：在分词前添加新 Token，使其优先被识别：
     ```python
     tokenizer.add_tokens(["天干"], special_tokens=False)
     ```

---

### **三、代码示例完整流程**
```python
from transformers import AutoTokenizer, AutoModel

# 1. 初始化分词器和模型
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = AutoModel.from_pretrained("bert-base-chinese")

# 2. 添加新 Token
new_tokens = ["天干", "地支"]
new_tokens = [token for token in new_tokens if token not in tokenizer.vocab]
tokenizer.add_tokens(new_tokens)

# 3. 调整模型嵌入层
model.resize_token_embeddings(len(tokenizer))

# 4. 验证
text = "天干地支用于纪年。"
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)

print(f"Tokenized: {tokenizer.tokenize(text)}")
# 输出: ['天干', '地支', '用', '于', '纪', '年', '。']
```

---

### **四、特殊场景处理**
#### **添加大量新 Token**
若需添加数百或数千个新 Token（如医学专业术语）：
1. **高效初始化嵌入**：  
   使用预训练词向量（如 Word2Vec）初始化新 Token 的嵌入，而非随机初始化。
   ```python
   # 示例：从外部文件加载嵌入向量
   import torch

   new_embeddings = load_external_embeddings()  # 自定义函数
   with torch.no_grad():
       model.embeddings.word_embeddings.weight[-len(new_tokens):, :] = torch.tensor(new_embeddings)
   ```

#### **动态添加 Token**
在运行时根据输入动态扩展词表（不推荐，可能破坏预训练权重的一致性）。

---

### **五、总结**
| 操作                | 方法                     | 是否需调整模型嵌入层 |
|---------------------|--------------------------|----------------------|
| 添加常规 Token       | `tokenizer.add_tokens()`  | 是                   |
| 添加特殊 Token       | `tokenizer.add_special_tokens()` | 是                   |
| 添加后处理子词冲突   | 优先添加长 Token          | 否                   |

通过合理添加新 Token 并调整模型，可以有效提升模型在特定领域的表现。建议在微调（Fine-tuning）过程中进一步优化新 Token 的嵌入表示。

## 为什么要加token呢？有什么用途？

在自然语言处理（NLP）任务中，**向分词器（Tokenizer）添加新 Token** 是优化模型性能、适应特定领域或任务需求的重要手段。以下是添加 Token 的核心原因和实际用途：

---

### **一、核心用途**
#### 1. **处理领域专有词汇**
   - **问题**：预训练模型的词表（Vocabulary）通常基于通用语料，可能缺少专业术语、新造词或领域专有词汇（如医学名词、科技术语）。
   - **示例**：  
     - 中文金融领域术语：“量化宽松”、“做市商”可能被拆分为子词（如“量化”+“宽松”），破坏语义完整性。
     - 英文生物学术语：“CRISPR-Cas9”可能被拆分为多个子词（如“CRI”、“##SP”、“##R-Cas9”）。
   - **解决方案**：将专有词汇作为独立 Token 添加到词表，确保模型将其视为整体。

#### 2. **提高分词效率**
   - **问题**：高频词被拆分为多个子词会增加输入长度，降低计算效率。
   - **示例**：  
     - 公司名“OpenAI”可能被拆分为“Open”+“A”+“I”，添加为独立 Token 后直接保留“OpenAI”。
   - **效果**：缩短序列长度，减少计算开销。

#### 3. **支持特殊符号或标记**
   - **需求**：任务中需要引入自定义标记（如领域分隔符、占位符）。
   - **示例**：  
     - 对话系统中的角色标记：`[用户]`、`[助手]`。
     - 多模态任务的图像占位符：`[图像]`。

#### 4. **优化模型对稀有词的处理**
   - **问题**：低频词可能被标记为 `[UNK]`（未知词），导致信息丢失。
   - **示例**：  
     - 中文古诗词中的生僻字（如“饕餮”）可能被拆解为笔画或错误处理。
   - **解决方案**：添加这些词为 Token，避免未知标记。

#### 5. **多语言混合场景**
   - **需求**：处理混合语言文本时，需支持非词表内字符。
   - **示例**：  
     - 中英文混合句子：“Python是一种高级编程语言。”
     - 添加“Python”为 Token，避免拆分为 `Py` + `##thon`。

---

### **二、实际应用场景**
#### 1. **领域适配（Domain Adaptation）**
   - **案例**：医疗领域的实体识别任务需要识别“冠状动脉粥样硬化”等术语，添加这些词可提升模型识别准确率。

#### 2. **品牌/产品名称保护**
   - **案例**：电商评论分析中，确保品牌名（如“iPhone 15 Pro”）不被拆分为多个 Token，保持语义连贯。

#### 3. **控制生成结果**
   - **案例**：在文本生成任务中，添加禁止词（如“[敏感词]”）为 Token，并通过掩码强制模型避开这些词。

#### 4. **支持新语言或方言**
   - **案例**：为方言词汇（如粤语“唔该”）添加 Token，帮助模型理解区域性表达。

#### 5. **结构化数据编码**
   - **案例**：将数字、日期格式（如“2023-09-15”）作为独立 Token，提升模型对结构化数据的处理能力。

---

### **三、技术优势**
| 操作                | 未添加新 Token 的缺陷                 | 添加新 Token 的优势                     |
|---------------------|--------------------------------------|----------------------------------------|
| **分词结果**         | 长词被拆分为多个子词，破坏语义完整性  | 关键术语保持完整，提升模型理解能力        |
| **输入长度**         | 序列长度增加，计算成本上升            | 缩短序列，提升推理速度                  |
| **模型性能**         | 未知词（[UNK]）导致信息丢失            | 避免未知词，保留完整信息                |
| **训练稳定性**       | 随机初始化的子词嵌入可能引入噪声        | 新 Token 嵌入可通过微调优化              |

---

### **四、操作验证示例**
#### 1. **添加前（子词拆分）**
```python
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
text = "量化宽松政策"
tokens = tokenizer.tokenize(text)  
print(tokens)  # 输出: ['量', '化', '宽', '松', '政', '策']
```

#### 2. **添加后（完整 Token）**
```python
tokenizer.add_tokens(["量化宽松"])
tokens = tokenizer.tokenize(text)
print(tokens)  # 输出: ['量化宽松', '政', '策']
```

---

### **五、注意事项**
1. **嵌入初始化**  
   添加新 Token 后需扩展模型的嵌入层，并合理初始化新 Token 的向量：
   - **默认**：随机初始化（可能需微调以适配任务）。
   - **高级策略**：使用已有词向量的平均值或外部预训练向量初始化。

2. **词表膨胀**  
   避免过度添加低频词，否则可能稀释模型容量，降低泛化性。

3. **跨模型兼容性**  
   同一 Token 在不同模型中的嵌入可能不兼容（如 BERT vs GPT）。

---

### **总结**
添加 Token 的核心目的是 **增强模型对特定任务或领域的适应性**，通过以下方式实现：
- 保留关键术语的语义完整性。
- 减少分词错误和未知词比例。
- 优化输入效率和模型性能。

这一操作在专业领域 NLP 任务（如医疗、金融、法律）和定制化应用中尤为关键。

## 添加的新token的意义？

以下是关于添加新 Token 对模型影响的详细分析：

---

### **一、添加新 Token 的本质**
#### 1. **词表扩展 ≠ 模型容量增加**
   - **模型容量**：由网络结构（如层数、隐藏层维度）决定，与词表大小无直接关系。
   - **词表扩展**：仅增加嵌入矩阵（Embedding Matrix）的行数，模型的其他参数（如注意力层、前馈层）保持不变。

#### 2. **新 Token 的初始化**
   - **默认行为**：新 Token 的嵌入向量（Embedding Vector）会被**随机初始化**，与预训练权重无关联。
   - **知识空白**：模型在预训练阶段从未见过这些 Token，因此初始状态下确实无法“理解”其语义。

---

### **二、如何让新 Token 变得有用？**
#### 1. **微调（Fine-tuning）的必要性**
   - **核心原理**：通过下游任务的数据训练，**调整新 Token 的嵌入向量**，使其融入模型的语义空间。
   - **示例**：  
     - 添加医学术语后，在医疗文本分类任务中微调，模型会学习到这些术语与标签（如“疾病类型”）的关联。

#### 2. **调整嵌入层**
   ```python
   model.resize_token_embeddings(len(tokenizer))  # 必须执行！
   ```
   - **作用**：扩展模型的嵌入层以匹配新词表，否则新 Token 会被映射到错误位置（如 `[UNK]`）。

#### 3. **嵌入初始化策略**
   - **随机初始化**（默认）：依赖后续微调学习有效表示。
   - **启发式初始化**（可选）：用已有词向量的平均值或外部嵌入初始化，加速收敛。
     ```python
     with torch.no_grad():
         new_embed = model.get_input_embeddings().weight[known_token_ids].mean(dim=0)
         model.get_input_embeddings().weight[new_token_id] = new_embed
     ```

---

### **三、新 Token 的有效性条件**
| 条件                | 有效性高                              | 有效性低                              |
|---------------------|-------------------------------------|-------------------------------------|
| **微调数据量**       | 大量包含新 Token 的标注数据           | 新 Token 在训练数据中极少出现         |
| **任务相关性**       | 新 Token 是任务的关键特征（如实体名） | 新 Token 与任务无关                   |
| **初始化质量**       | 使用合理的嵌入初始化                  | 完全随机初始化且微调不足               |
| **模型容量**         | 模型足够大（如 BERT-large）           | 模型过小（如 TinyBERT）               |

---

### **四、实际场景验证**
#### **案例：添加电商品牌名**
1. **未微调**  
   - 模型无法区分新品牌名（如“VIVOX90”）和普通词（如“手机”），生成或分类结果可能混乱。
2. **微调后**  
   - 模型通过训练数据学习到“VIVOX90”与“手机”、“旗舰机型”等词的关联，提升任务表现。

#### **代码验证（微调前后对比）**
```python
# 微调前：新 Token 的相似度随机
from scipy.spatial.distance import cosine

embeddings = model.get_input_embeddings().weight
token_id = tokenizer.convert_tokens_to_ids("VIVOX90")
similarities = [1 - cosine(embeddings[token_id], embeddings[i]) for i in range(100)]
print("Top 5 similar words:", tokenizer.convert_ids_to_tokens(np.argsort(similarities)[-5:]))
# 输出可能为无关词：['的', '是', '和', '在', '##吗']

# 微调后重新检查
# 输出可能为相关词：['手机', '旗舰', 'Pro', 'X90', 'VIVO']
```

---

### **五、何时无需添加新 Token？**
1. **子词组合已足够**  
   若新词可被合理拆分为子词（如“机器学习”→“机器”+“学习”），且任务不依赖完整语义，无需添加。
2. **数据极度匮乏**  
   添加 Token 但缺乏训练数据时，可能引入噪声，降低模型性能。

---

### **六、总结**
- **添加新 Token** 本质是为模型提供**潜在的特征空间扩展机会**，而非直接提升容量。
- **实际价值** 取决于：
  - 后续微调是否充分（数据量、训练轮数）。
  - 新 Token 与任务目标的相关性。
- **优化建议**：
  1. **必做**：添加后执行 `resize_token_embeddings`。
  2. **推荐**：用领域数据微调模型，使新 Token 嵌入适应任务。
  3. **可选**：合理初始化新 Token 嵌入以加速训练。

通过科学添加和有效微调，新 Token 可以显著提升模型在特定场景下的表现，但需避免盲目添加导致的过拟合或噪声干扰。

## 也就是说，原本大模型不认识某个词，我现在告诉它这个词是什么，还需要进行微调，让它理解这个词？

你的理解基本正确，但需要更准确地拆解技术细节。让我们通过一个类比来明确这一过程：

---

### **一、核心逻辑类比**
想象大模型是一个经验丰富的语言学家，但只懂通用领域的词汇。现在你想让它理解某个专业术语（比如“量子纠缠”），以下是分步过程：
1. **添加术语到词典**（`tokenizer.add_tokens()`）  
   - 相当于给语言学家一本新词典，里面包含这个词，但他从未学过这个词的实际含义。
2. **微调（Fine-tuning）**  
   - 相当于让语言学家阅读大量包含“量子纠缠”的文献（领域数据），通过上下文学习这个词的用法和关联概念。

**关键结论**：  
- 仅添加 Token 到词表（扩展词典）无法让模型理解词义。
- 必须通过微调，让模型在具体任务中学习新 Token 的语义和上下文关系。

---

### **二、技术细节拆解**
#### 1. **添加 Token 的作用**
   - **分词正确性**：确保新词不被拆分为子词（如“量子纠缠” → “量”+“子”+“纠”+“缠”）。
   - **输入完整性**：模型接收到的输入是完整词汇而非碎片。
   - **嵌入可调性**：为新词分配可训练的嵌入向量。

#### 2. **未微调的局限性**
   - **随机嵌入**：新 Token 的向量初始化是随机的，与预训练语义空间不兼容。
   - **语义断层**：模型无法通过注意力机制关联新 Token 与已有知识。
   - **输出不可控**：生成或分类时，新 Token 可能产生不合理结果。

#### 3. **微调的核心价值**
   - **嵌入调整**：通过反向传播优化新 Token 的向量，使其融入模型的语义空间。
   - **上下文学习**：让模型理解新 Token 在具体任务中的用法（如“量子纠缠”常与“叠加态”共现）。

---

### **三、操作步骤与验证**
#### **1. 添加新 Token 并微调**
```python
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# 加载模型和分词器
model_name = "bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 添加新 Token
new_tokens = ["量子纠缠"]
tokenizer.add_tokens(new_tokens)
model.resize_token_embeddings(len(tokenizer))  # 必须调整嵌入层！

# 准备微调数据（示例）
train_texts = ["量子纠缠是量子力学的核心现象", "实验验证了量子纠缠的超距作用"]
train_labels = [1, 1]  # 假设为二分类任务

# 微调循环
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=...  # 需封装为 Dataset 格式
)

trainer.train()
```

#### **2. 微调效果验证**
- **嵌入相似度变化**：
  ```python
  # 微调前
  emb_before = model.bert.embeddings.word_embeddings.weight[-1]  # 新 Token 的嵌入
  
  # 微调后
  emb_after = model.bert.embeddings.word_embeddings.weight[-1]
  
  # 检查与相关词的相似度变化
  particle_id = tokenizer.convert_tokens_to_ids("粒子")
  emb_particle = model.bert.embeddings.word_embeddings.weight[particle_id]
  
  print("微调前相似度:", 1 - cosine(emb_before, emb_particle))  # 可能接近0（随机）
  print("微调后相似度:", 1 - cosine(emb_after, emb_particle))   # 应显著提高
  ```

---

### **四、何时可以跳过微调？**
| 场景                          | 是否需微调 | 原因                                                                 |
|------------------------------|------------|----------------------------------------------------------------------|
| **新词可合理拆分为子词**       | 否         | 模型已通过子词理解语义（如“量子计算机” → “量子”+“计算机”）            |
| **新词仅用于输入，不参与输出** | 否         | 模型只需读取该词，无需生成或分类（如搜索索引中的关键词）               |
| **新词为低频且非关键术语**     | 否         | 对任务影响可忽略                                                   |
| **新词为核心任务特征**         | 是         | 必须通过微调建立其与标签的关联                                       |

---

### **五、最佳实践**
1. **数据质量优先**：确保微调数据充分包含新 Token 的多种上下文。
2. **渐进式扩展**：分批添加关键 Token 并逐步微调，避免一次性引入过多噪声。
3. **监控过拟合**：添加过多低频 Token 可能导致模型记忆训练数据而非泛化。

---

### **总结**
- **添加 Token** = 让模型“认识”这个词的存在。
- **微调** = 让模型“理解”这个词的语义和用法。
- 两者缺一不可，才能有效提升模型在特定领域的表现。

In [59]:
len(tokenizer.vocab.keys())

21130

In [60]:
from itertools import islice

# 使用 islice 查看词表部分内容
for key, value in islice(tokenizer.vocab.items(), 10):
    print(f"{key}: {value}")

##涟: 16935
經: 5195
##拾: 15953
蚝: 6016
##cc: 8860
哼: 1530
##吮: 14479
price: 11597
鈉: 7042
霍: 7452


In [61]:
new_tokens = ["天干", "地支"]

In [62]:
# 将集合作差结果添加到词表中
new_tokens = set(new_tokens) - set(tokenizer.vocab.keys())

In [63]:
new_tokens

set()

In [64]:
tokenizer.add_tokens(list(new_tokens))

0

In [65]:
# 新增加了2个Token，词表总数由 21128 增加到 21130
len(tokenizer.vocab.keys())

21130

#### 添加特殊Token（审慎操作）

---

### 分步解释与注意事项

在 Hugging Face Transformers 库中，使用 `add_special_tokens` 方法添加特殊 Token 时，词表的变化可能涉及多个隐藏机制。以下是关于你操作的分步解析：

---

#### **1. 添加特殊 Token 的代码逻辑**
```python
new_special_token = {"sep_token": "NEW_SPECIAL_TOKEN"}
tokenizer.add_special_tokens(new_special_token)
```
- **功能**：将 `NEW_SPECIAL_TOKEN` 设置为新的分隔符（`sep_token`）。
- **预期行为**：
  - 若 `sep_token` 原本不存在于词表中，直接添加该 Token。
  - 若 `sep_token` 已存在（如 BERT 默认的 `[SEP]`），则 **替换原 Token 的功能**，但原 Token 可能仍保留在词表中。

---

#### **2. 词表大小变化的可能原因**
- **现象**：词表从 `21128` → `21131`（增加 3）。
- **可能原因**：
  1. **替换原有 `sep_token`**  
     若原始 `sep_token` 是 `[SEP]`（ID 102），替换后：
     - `[SEP]` 的 **功能被禁用**（不再是分隔符），但它仍保留在词表中。
     - `NEW_SPECIAL_TOKEN` 作为新 `sep_token` 被添加到词表。
     - **词表总数增加 1**（仅新 Token）。
  2. **自动扩展 `additional_special_tokens`**  
     某些模型（如 BERT）要求特殊 Token 必须位于 `additional_special_tokens` 列表中。当添加新 Token 时，库可能自动将其附加到该列表，导致 **列表扩容**。
  3. **分词器内部处理**  
     某些分词器（如基于 SentencePiece 的模型）在添加特殊 Token 时，可能自动插入辅助符号，导致多个隐式 Token 被添加。

---

#### **3. 验证实际变化**
- **步骤 1：检查特殊 Token 映射**  
  ```python
  print(tokenizer.sep_token)          # 输出: "NEW_SPECIAL_TOKEN"
  print(tokenizer.sep_token_id)       # 输出: 21129（假设新增 ID）
  ```
- **步骤 2：查看原始 `[SEP]` 是否存在**  
  ```python
  print(tokenizer.vocab["[SEP]"])     # 输出: 102（若存在）
  ```
- **步骤 3：检查 `additional_special_tokens`**  
  ```python
  print(tokenizer.additional_special_tokens)
  # 可能的输出: ["NEW_SPECIAL_TOKEN"]
  ```

---

#### **4. 为何词表增加 3？**
根据你的代码，**仅添加了 1 个新 Token**，但词表增加 3，可能原因：
1. **误操作**：在运行此代码前，可能已执行过其他添加 Token 的操作（如 `add_tokens()`）。
2. **分词器类型差异**：  
   - 若使用 **GPT-2** 或 **RoBERTa** 等模型，其分词器可能将 `NEW_SPECIAL_TOKEN` 拆分为多个子词（如 `_NEW`, `_SPECIAL`, `_TOKEN`），导致词表增加多个条目。
   - 但 `add_special_tokens` 方法通常会将 Token **强制添加为独立单元**，不进行拆分。
3. **模型配置冲突**：某些模型（如 ALBERT）的 Tokenizer 可能因内部约束自动添加辅助 Token。

---

### 正确操作与建议

#### **1. 安全添加特殊 Token 的流程**
```python
# 添加前：检查现有特殊 Token
print(tokenizer.special_tokens_map)
# 输出示例: {'sep_token': '[SEP]', 'pad_token': '[PAD]', ...}

# 添加新特殊 Token（替换 sep_token）
new_special_token = {"sep_token": "NEW_SPECIAL_TOKEN"}
tokenizer.add_special_tokens(new_special_token)

# 验证添加结果
print(tokenizer.sep_token)          # 输出: "NEW_SPECIAL_TOKEN"
print(tokenizer.special_tokens_map) # 检查 sep_token 是否更新
```

#### **2. 处理词表变化的注意事项**
- **强制单 Token 添加**：  
  若需确保新 Token 不被拆分，使用 `add_special_tokens` 而非 `add_tokens`。
- **模型嵌入层调整**：  
  添加后必须调整模型嵌入大小：
  ```python
  model.resize_token_embeddings(len(tokenizer))
  ```
- **清理历史操作**：  
  若词表变化不符合预期，重新初始化 Tokenizer 并逐步验证操作。

---

### 总结
- **词表增加 3 的异常**：很可能是代码执行环境中的历史操作残留（如之前未显示的 `add_tokens` 调用），而非当前代码导致。
- **核心原则**：  
  - 使用 `add_special_tokens` 添加的特殊 Token **不应被拆分**。
  - 每次操作后需验证 `vocab` 和 `special_tokens_map` 的实际变化。

## 为什么要添加特殊token？

在自然语言处理任务中，**添加特殊 Token** 的主要目的是为了 **增强模型对特定任务或领域输入的语义理解和结构化处理能力**。以下是详细解释其必要性和应用场景：

---

### 一、特殊 Token 的核心作用
#### 1. **结构化输入**  
特殊 Token 为模型提供 **结构化信号**，帮助其识别输入的不同部分。例如：
- **分隔句子**：`[SEP]` 用于区分多个句子（如问答任务中的问题和上下文）。
- **分类标识**：`[CLS]` 用于分类任务的特征聚合。
- **角色标记**：在对话系统中使用 `[用户]`、`[助手]` 区分不同发言者。

#### 2. **控制模型行为**  
通过特殊 Token 触发特定生成逻辑：
- **终止生成**：`[EOS]`（End-of-Sequence）标记终止文本生成。
- **掩码预测**：`[MASK]` 用于 BERT 的完形填空任务。
- **领域引导**：添加 `[医学]`、`[法律]` 等标记，引导模型聚焦特定领域。

#### 3. **处理未见内容**  
为模型引入 **自定义语义单元**，避免未知词（`[UNK]`）导致的信息丢失：
- **领域术语**：如添加 `[DNA]` 表示基因序列中的特定片段。
- **特殊符号**：如数学公式中的 `[公式]` 占位符。

---

### 二、何时需要添加特殊 Token？
| 场景                  | 需添加的特殊 Token        | 作用示例                     |
|-----------------------|--------------------------|-----------------------------|
| **多轮对话**           | `[用户]`、`[助手]`          | 区分对话角色                  |
| **文本生成控制**       | `[正面情感]`、`[负面情感]`   | 控制生成文本的情感倾向         |
| **结构化数据输入**     | `[表格]`、`[标题]`          | 标识输入中的结构化元素         |
| **跨模态任务**         | `[图像]`、`[音频]`          | 提示模型处理多模态输入         |
| **领域适配**           | `[医学实体]`、`[法律条款]`   | 提升领域术语的识别能力         |

---

### 三、技术实现原理
#### 1. **分词器与模型的协作**  
- **分词器**：将特殊 Token 视为独立单元，不进行子词拆分。
- **模型**：
  - **嵌入层**：为特殊 Token 分配独立的嵌入向量。
  - **注意力机制**：模型通过预训练或微调学习这些 Token 的上下文关联。

#### 2. **添加步骤示例**  
```python
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

# 添加特殊 Token：医学实体标记
special_tokens = {"additional_special_tokens": ["[医学实体]"]}
tokenizer.add_special_tokens(special_tokens)

# 验证添加结果
print(tokenizer.additional_special_tokens)  # 输出: ['[医学实体]']
print(tokenizer("[医学实体] 患者出现发热症状")["input_ids"])
# 输出: [101, 21128, 704, 1744, ..., 102] （假设 21128 是新 Token ID）
```

#### 3. **必须调整模型嵌入层**  
```python
model.resize_token_embeddings(len(tokenizer))  # 同步扩展模型的嵌入矩阵
```

---

### 四、实战案例：医疗实体识别
#### **目标**  
提升模型对医疗文本中实体（如疾病、症状）的识别能力。

#### **步骤**  
1. **添加特殊 Token**  
   ```python
   special_tokens = {
       "additional_special_tokens": ["[疾病]", "[症状]", "[药品]"]
   }
   tokenizer.add_special_tokens(special_tokens)
   ```

2. **微调模型**  
   使用标注数据训练模型，使其学习到：
   - `[疾病]` 后接疾病名称（如“糖尿病”）。
   - `[症状]` 后接症状描述（如“头痛”）。

3. **推理效果**  
   ```python
   text = "患者主诉[症状] 持续性头痛 [疾病] 高血压"
   inputs = tokenizer(text, return_tensors="pt")
   outputs = model(**inputs)
   # 模型可识别出 "持续性头痛" 属于症状，"高血压" 属于疾病
   ```

---

### 五、注意事项
#### 1. **谨慎选择 Token 数量**  
- **少量添加**：特殊 Token 过多会稀释模型对常规文本的关注。
- **语义明确**：确保 Token 名称直观反映其功能（如 `[摘要]` 比 `[特殊标记1]` 更易理解）。

#### 2. **数据需包含新 Token**  
- **训练数据**：微调数据中需充分包含新 Token 的上下文示例。
- **分布合理**：避免 Token 集中在少数样本中导致过拟合。

#### 3. **初始化策略**  
- **随机初始化**：默认方式，依赖微调学习。
- **启发式初始化**：用相似语义 Token 的嵌入均值加速收敛。

---

### 六、总结
添加特殊 Token 是 **领域适配和任务定制化** 的关键技术，通过：
1. **结构化输入**：明确任务需求。
2. **语义控制**：引导模型生成或理解特定内容。
3. **信息完整性**：避免关键内容被错误拆分。

配合微调，模型能有效学习新 Token 的语义，从而在专业场景中显著提升表现。

## 特殊token，就是说要染大模型关注某些有特殊意义的token，但同时要注意不要随意覆盖原有的某些token？


In [66]:
new_special_token = {"sep_token": "NEW_SPECIAL_TOKEN"}

In [67]:
tokenizer.add_special_tokens(new_special_token)

1

In [68]:
# 新增加了1个特殊Token，词表总数由 21128 增加到 21131
len(tokenizer.vocab.keys())

21131

### 使用 `save_pretrained` 方法保存指定 Model 和 Tokenizer 

借助 `AutoClass` 的设计理念，保存 Model 和 Tokenizer 的方法也相当高效便捷。

假设我们对`bert-base-chinese`模型以及对应的 `tokenizer` 做了修改，并更名为`new-bert-base-chinese`，方法如下：

```python
tokenizer.save_pretrained("./models/new-bert-base-chinese")
model.save_pretrained("./models/new-bert-base-chinese")
```

保存 Tokenizer 会在指定路径下创建以下文件：
- tokenizer.json: Tokenizer 元数据文件；
- special_tokens_map.json: 特殊字符映射关系配置文件；
- tokenizer_config.json: Tokenizer 基础配置文件，存储构建 Tokenizer 需要的参数；
- vocab.txt: 词表文件；
- added_tokens.json: 单独存放新增 Tokens 的配置文件。

保存 Model 会在指定路径下创建以下文件：
- config.json：模型配置文件，存储模型结构参数，例如 Transformer 层数、特征空间维度等；
- pytorch_model.bin：又称为 state dictionary，存储模型的权重。

以下是使用 `save_pretrained` 方法保存修改后的模型和分词器的完整说明，包含操作步骤、文件解析及注意事项：

---

### **操作步骤**
#### **1. 修改模型和分词器**
```python
from transformers import AutoTokenizer, AutoModel

# 加载原始模型和分词器
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = AutoModel.from_pretrained("bert-base-chinese")

# 添加新 Token（例如领域术语）
new_tokens = ["天干", "地支"]
tokenizer.add_tokens(new_tokens)
model.resize_token_embeddings(len(tokenizer))  # 必须调整模型嵌入层！

# 添加特殊 Token（如领域分隔符）
special_tokens = {"additional_special_tokens": ["[医学]"]}
tokenizer.add_special_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))  # 再次调整嵌入层
```

#### **2. 保存到指定目录**
```python
save_path = "./models/new-bert-base-chinese"

# 保存分词器
tokenizer.save_pretrained(save_path)  
# 输出文件列表：
# - tokenizer_config.json
# - special_tokens_map.json
# - vocab.txt
# - added_tokens.json
# - tokenizer.json

# 保存模型
model.save_pretrained(save_path)  
# 输出文件列表：
# - config.json
# - pytorch_model.bin
```

---

### **生成文件解析**
#### **Tokenizer 文件**
| 文件名                  | 作用                                                                 |
|-------------------------|----------------------------------------------------------------------|
| `tokenizer_config.json`  | 分词器配置参数（如模型类型、填充方向、特殊 Token 定义）。            |
| `special_tokens_map.json` | 所有特殊 Token 的映射（如 `cls_token`、`sep_token`）。               |
| `vocab.txt`             | 基础词表（原始词汇 + 新增常规 Token）。                              |
| `added_tokens.json`     | **新增的 Token**（包括通过 `add_tokens` 和 `add_special_tokens` 添加的）。 |
| `tokenizer.json`        | 分词器的完整序列化数据（供 `from_pretrained` 快速加载）。              |

#### **Model 文件**
| 文件名               | 作用                                                                 |
|----------------------|----------------------------------------------------------------------|
| `config.json`        | 模型结构配置（如隐藏层维度、注意力头数、Transformer 层数）。          |
| `pytorch_model.bin`  | 模型权重（包含调整后的嵌入层权重和所有预训练参数）。                  |

---

### **加载验证**
```python
# 重新加载验证
new_tokenizer = AutoTokenizer.from_pretrained(save_path)
new_model = AutoModel.from_pretrained(save_path)

# 测试新 Token 是否生效
text = "[医学]患者出现天干地支症状"
inputs = new_tokenizer(text, return_tensors="pt")
outputs = new_model(**inputs)

print(new_tokenizer.tokenize(text))
# 预期输出：['[医学]', '患', '者', '出', '现', '天干', '地支', '症', '状']
```

---

### **注意事项**
#### **1. 必须调整模型嵌入层**
- **原因**：添加新 Token 后，模型的嵌入矩阵需扩展以匹配新词表大小。
- **操作**：在保存前调用 `model.resize_token_embeddings(len(tokenizer))`。
- **后果**：若未调整，加载时会因维度不匹配报错（如 `size mismatch`）。

#### **2. 避免覆盖核心 Token**
- **安全操作**：通过 `add_special_tokens` 添加而非直接修改原有 Token（如 `[CLS]`、`[SEP]`）。
- **验证方法**：
  ```python
  print(new_tokenizer.sep_token)  # 应为原始 [SEP]，而非自定义 Token
  ```

#### **3. 文件完整性**
- **必需文件**：确保保存目录包含所有生成文件，缺失任一文件将导致加载失败。
- **手动修改风险**：避免直接编辑 JSON 或词表文件，可能导致序列化错误。

#### **4. 跨框架兼容性**
- **PyTorch 转 TensorFlow**：需额外调用 `TFModel.from_pretrained(save_path, from_pt=True)`。
- **设备映射**：加载时指定 `device_map="auto"` 可自动分配 GPU/CPU。

#### **5. 版本控制**
- **库版本**：不同版本的 `transformers` 可能导致配置格式差异，建议保存和加载时使用相同版本。
- **模型兼容性**：修改后的模型仅兼容相同架构的加载（如 `BertModel` 不可加载为 `RobertaModel`）。

---

### **总结**
通过 `save_pretrained` 方法，可以高效保存自定义的模型和分词器，确保所有修改（如新增 Token、调整配置）被完整记录。关键操作步骤包括：
1. **修改后调整嵌入层**：`model.resize_token_embeddings()`。
2. **完整保存配置和权重**：使用标准方法生成文件。
3. **验证加载结果**：确保新 Token 和模型行为符合预期。

这种流程使得领域适配、任务定制化开发更加便捷，同时保障了模型的可复现性和部署稳定性。

In [69]:
tokenizer.save_pretrained("./models/new-bert-base-chinese")

('./models/new-bert-base-chinese/tokenizer_config.json',
 './models/new-bert-base-chinese/special_tokens_map.json',
 './models/new-bert-base-chinese/vocab.txt',
 './models/new-bert-base-chinese/added_tokens.json',
 './models/new-bert-base-chinese/tokenizer.json')

In [58]:
model.save_pretrained("./models/new-bert-base-chinese")